# **Multiple Linear Regression**

## _Machine Learning Modelling Workflow Notebook 1_

## Content:
1. Splitting the dataframe into train and test subsets;
2. Ordinary Least Squares (OLS) Linear Regression;
3. Ridge Linear Regression;
4. Lasso Linear Regression;
5. Elastic Net Linear Regression;
6. Getting a general feature ranking;
7. Calculating metrics for regression models;
8. Making predictions with the models.

Marco Cesar Prado Soares, Data Scientist Specialist - Bayer Crop Science LATAM
- marcosoares.feq@gmail.com
- marco.soares@bayer.com

Install statsmodels library

In [ ]:
! pip install statsmodels

Install tensorflow library

In [ ]:
! pip install tensorflow

Install Keras library

In [ ]:
! pip install keras

Install SHAP library

In [ ]:
! pip install shap

In [ ]:
#check the version of the package
! pip show shap

In [ ]:
# Upgrade to the most recent library versions, if a given module is not present and analysis cannot be
# executed.
! pip install pip --upgrade
! pip install tensorflow --upgrade
! pip install keras --upgrade
! pip install shap --upgrade
! pip install sklearn --upgrade
! pip install pandas --upgrade
! pip install numpy --upgrade
! pip install matplotlib --upgrade
! pip install seaborn --upgrade
! pip install scipy --upgrade
! pip install statsmodels --upgrade

## **Load Python Libraries in Global Context**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels as sm
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from keras.layers import Dense
from keras.models import Sequential
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

# **Function for mounting Google Drive or S3 (AWS Simple Storage Service) bucket**

In [2]:
def mount_storage_system (source = 'aws', path_to_store_imported_s3_bucket = '/', s3_bucket_name = None, s3_obj_key_preffix = None):
    
    import sagemaker
    # sagemaker is AWS SageMaker Python SDK
    from sagemaker.session import Session
    from google.colab import drive
    
    # source = 'google' for mounting the google drive;
    # source = 'aws' for mounting an AWS S3 bucket.
    
    # THE FOLLOWING PARAMETERS HAVE EFFECT ONLY WHEN source == 'aws'
    
    # path_to_store_imported_s3_bucket: path of the Python environment to which the
    # S3 bucket contents will be imported. If it is None, or if 
    # path_to_store_imported_s3_bucket = '/', bucket will be imported to the root path. 
    # Alternatively, input the path as a string (in quotes). e.g. 
    # path_to_store_imported_s3_bucket = '/copied_s3_bucket'
    
    # s3_bucket_name = None.
    ## This parameter is obbligatory to access an AWS S3 bucket. Substitute it for a string
    # with the bucket's name. e.g. s3_bucket_name = "aws-bucket-1" access a bucket named as
    # "aws-bucket-1"
    
    # s3_obj_key_preffix = None. Keep it None or as an empty string (s3_obj_key_preffix = '')
    # to import the whole bucket content, instead of a single object from it.
    # Alternatively, set it as a string containing the subfolder from the bucket to import:
    # Suppose that your bucket (admin-created) has four objects with the following object 
    # keys: Development/Projects1.xls; Finance/statement1.pdf; Private/taxdocument.pdf; and
    # s3-dg.pdf. The s3-dg.pdf key does not have a prefix, so its object appears directly 
    # at the root level of the bucket. If you open the Development/ folder, you see 
    # the Projects.xlsx object in it.
    # Check Amazon documentation:
    # https://docs.aws.amazon.com/AmazonS3/latest/userguide/object-keys.html
    
    # In summary, if the path of the file is: 'bucket/my_path/.../file.csv'
    # where 'bucket' is the bucket's name, key_preffix = 'my_path/.../', without the
    # 'file.csv' (file name with extension) last part.
    
    if (source == 'google'):
        
        print("Associate the Python environment to your Google Drive account, and authorize the access in the opened window.")
        
        drive.mount('/content/drive')
        
        print("Now your Python environment is connected to your Google Drive: the root directory of your environment is now the root of your Google Drive.")
        print("In Google Colab, navigate to the folder icon (\'Files\') of the left navigation menu to find a specific folder or file in your Google Drive.")
        print("Click on the folder or file name and select the elipsis (...) icon on the right of the name to reveal the option \'Copy path\', which will give you the path to use as input for loading objects and files on your Python environment.")
        print("Caution: save your files into different directories of the Google Drive. If files are all saved in a same folder or directory, like the root path, they may not be accessible from your Python environment.")
        print("If you still cannot see the file after moving it to a different folder, reload the environment.")
    
    elif (source == 'aws'):
        
        # Notice: if you wanted to authenticate directly from Python code, you could use
        # the following code, instead, to start the S3 client. boto3 is AWS S3 Python SDK:
        
        # import boto3
        # ACCESS_KEY = 'access_key_ID'
        # PASSWORD_KEY = 'password_key'
        # s3_client = boto3.client('s3', aws_access_key_id = ACCESS_KEY, aws_secret_access_key = PASSWORD_KEY)
        # ... [here, use the same following code until line new_session = Session()]
        # [keep the line for session start. Substitute the line with the .download_data
        # method by the following line:]
        # s3_client.download_file(s3_bucket_name, s3_file_name_with_extension, path_to_store_imported_s3_bucket)
        
        # Check if the whole bucket will be downloaded (s3_obj_key_preffix = None):
        if (s3_obj_key_preffix is None):
            
            s3_obj_key_preffix = ''
        
        # If the path to store is None, also import the bucket to the root path:
        if (path_to_store_imported_s3_bucket is None):
            
            path_to_store_imported_s3_bucket = '/'
        
        # If the bucket name was provided, start the session. If not, print an error
        # message:
        
        if (s3_bucket_name is None):
            
            print("Please, provide a valid S3 Bucket name to download from.")
        
        else:
        
            # start a new sagemaker session:

            print("Starting a SageMaker session to be associated with the S3 bucket.")

            new_session = Session()
            # Check sagemaker session class documentation:
            # https://sagemaker.readthedocs.io/en/stable/api/utility/session.html
            session.download_data(path = path_to_store_imported_s3_bucket, bucket = s3_bucket_name, key_prefix = s3_obj_key_preffix)

            print(f"S3 bucket contents successfully imported to path \'{path_to_store_imported_s3_bucket}\'.")
            
    else:
        
        print("Select a valid source: \'google\' for mounting Google Drive; or \'aws\' for accessing AWS S3 Bucket.")

# **Function for loading the dataframe**

In [13]:
def load_dataframe (file_directory_path, file_name_with_extension, load_txt_file_with_json_format = False, has_header = True, txt_csv_col_sep = "comma", sheet_to_load = None, json_record_path = None, json_field_separator = "_", json_metadata_preffix_list = None):
    
    import os
    import json
    import pandas as pd
    from pandas import json_normalize
    
    ## WARNING: Use this function to load dataframes stored on Excel (xls, xlsx, etc), 
    ## JSON, txt, or CSV (comma separated values) files.
    
    # file_directory_path - (string, in quotes): input the path of the directory (e.g. folder path) 
    # where the file is stored. e.g. file_directory_path = "/" or file_directory_path = "/folder"
    
    # file_name_with_extension - (string, in quotes): input the name of the file with the extension
    # e.g. file_name_with_extension = "file.xlsx", or, file_name_with_extension = "file.csv"
    
    # load_txt_file_with_json_format = False. Set load_txt_file_with_json_format = True 
    # if you want to read a file with txt extension containing a text formatted as JSON 
    # (but not saved as JSON).
    # WARNING: if load_txt_file_with_json_format = True, all the JSON file parameters of the 
    # function (below) must be set. If not, an error message will be raised.
    
    # has_header = True if the the imported table has headers (row with columns names).
    # Alternatively, has_header = False if the dataframe does not have header.
    
    ## Parameters for loading txt or CSV files
    
    # txt_csv_col_sep = "comma" - This parameter has effect only when the file is a 'txt'
    # or 'csv'. It informs how the different columns are separated.
    # Alternatively, txt_csv_col_sep = "comma" for columns separated by comma (",")
    # txt_csv_col_sep = "whitespace" for columns separated by simple spaces (" ").
    
    # sheet_to_load - This parameter has effect only when for Excel files.
    # keep sheet_to_load = None not to specify a sheet of the file, so that the first sheet
    # will be loaded.
    # sheet_to_load may be an integer or an string (inside quotes). sheet_to_load = 0
    # loads the first sheet (sheet with index 0); sheet_to_load = 1 loads the second sheet
    # of the file (index 1); sheet_to_load = "Sheet1" loads a sheet named as "Sheet1".
    # Declare a number to load the sheet with that index, starting from 0; or declare a
    # name to load the sheet with that name.
    
    ## Parameters for loading JSON files:
    
    # https://docs.python.org/3/library/json.html
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html#pandas.json_normalize
    
    # json_record_path (string): manipulate parameter 'record_path' from json_normalize method.
    # Path in each object to list of records. If not passed, data will be assumed to 
    # be an array of records. If a given field from the JSON stores a nested JSON (or a nested
    # dictionary) declare it here to decompose the content of the nested data. e.g. if the field
    # 'books' stores a nested JSON, declare, json_record_path = 'books'
    
    # json_field_separator = "_" (string). Manipulates the parameter 'sep' from json_normalize method.
    # Nested records will generate names separated by sep. 
    # e.g., for json_field_separator = ".", {‘foo’: {‘bar’: 0}} -> foo.bar.
    # Then, if a given field 'main_field' stores a nested JSON with fields 'field1', 'field2', ...
    # the name of the columns of the dataframe will be formed by concatenating 'main_field', the
    # separator, and the names of the nested fields: 'main_field_field1', 'main_field_field2',...
    
    # json_metadata_preffix_list: list of strings (in quotes). Manipulates the parameter 
    # 'meta' from json_normalize method. Fields to use as metadata for each record in resulting 
    # table. Declare here the non-nested fields, i.e., the fields in the principal JSON. They
    # will be repeated in the rows of the dataframe to give the metadata (context) of the rows.
    
    # e.g. Suppose a JSON with the following structure: {'name': 'Mary', 'last': 'Shelley',
    # 'books': [{'title': 'Frankestein', 'year': 1818}, {'title': 'Mathilda ', 'year': 1819},{'title': 'The Last Man', 'year': 1826}]},
    # Here, there are nested JSONs in the field 'books'. The fields that are not nested
    # are 'name' and 'last'.
    # Then, json_record_path = 'books'
    # json_metadata_preffix_list = ['name', 'last']
    
    # Create the complete file path:
    file_path = os.path.join(file_directory_path, file_name_with_extension)
    # Extract the file extension
    file_extension = os.path.splitext(file_path)[1][1:]
    # os.path.splitext(file_path) is a tuple of strings: the first is the complete file
    # root with no extension; the second is the extension starting with a point: '.txt'
    # When we set os.path.splitext(file_path)[1], we are selecting the second element of
    # the tuple. By selecting os.path.splitext(file_path)[1][1:], we are taking this string
    # from the second character (index 1), eliminating the dot: 'txt'
    
    if ((file_extension == 'txt') | (file_extension == 'csv')): 
        # The operator & is equivalent to 'And' (intersection).
        # The operator | is equivalent to 'Or' (union).
        # pandas.read_csv method must be used.
        if (load_txt_file_with_json_format == True):
            
            print("Reading a txt file containing JSON parsed data. A reading error will be raised if you did not set the JSON parameters.")
            
            with open(file_path, 'r') as opened_file:
                # 'r' stands for read mode; 'w' stands for write mode
                # read the whole file as a string named 'file_full_text'
                file_full_text = opened_file.read()
                # if we used the readlines() method, we would be reading the
                # file by line, not the whole text at once.
                # https://stackoverflow.com/questions/8369219/how-to-read-a-text-file-into-a-string-variable-and-strip-newlines?msclkid=a772c37bbfe811ec9a314e3629df4e1e
                # https://www.tutorialkart.com/python/python-read-file-as-string/#:~:text=example.py%20%E2%80%93%20Python%20Program.%20%23open%20text%20file%20in,and%20prints%20it%20to%20the%20standard%20output.%20Output.?msclkid=a7723a1abfe811ecb68bba01a2b85bd8
                
            #Now, file_full_text is a string containing the full content of the txt file.
            json_file = json.loads(file_full_text)
            # json.load() : This method is used to parse JSON from URL or file.
            # json.loads(): This method is used to parse string with JSON content.
            # e.g. .json.loads() must be used to read a string with JSON and convert it to a flat file
            # like a dataframe.
            # check: https://www.pythonpip.com/python-tutorials/how-to-load-json-file-using-python/#:~:text=The%20json.load%20%28%29%20is%20used%20to%20read%20the,and%20alter%20data%20in%20our%20application%20or%20system.
            dataset = json_normalize(json_file, record_path = json_record_path, sep = json_field_separator, meta = json_metadata_preffix_list)
        
        else:
            # Not a JSON txt
        
            if (has_header == True):

                if (txt_csv_col_sep == "comma"):

                    dataset = pd.read_csv(file_path)

                elif (txt_csv_col_sep == "whitespace"):

                    dataset = pd.read_csv(file_path, delim_whitespace = True)

                else:
                    print(f"Enter a valid column separator for the {file_extension} file: \'comma\' or \'whitespace\'.")

            else:
                # has_header == False

                if (txt_csv_col_sep == "comma"):

                    dataset = pd.read_csv(file_path, header = None)

                elif (txt_csv_col_sep == "whitespace"):

                    dataset = pd.read_csv(file_path, delim_whitespace = True, header = None)

                else:
                    print(f"Enter a valid column separator for the {file_extension} file: \'comma\' or \'whitespace\'.")

    elif (file_extension == 'json'):
        
        with open(file_path) as opened_file:
            
            json_file = json.load(opened_file)
            # The structure json_file = json.load(open(file_path)) relies on the GC to close the file. That's not a 
            # good idea: If someone doesn't use CPython the garbage collector might not be using refcounting (which 
            # collects unreferenced objects immediately) but e.g. collect garbage only after some time.
            # Since file handles are closed when the associated object is garbage collected or closed 
            # explicitly (.close() or .__exit__() from a context manager) the file will remain open until 
            # the GC kicks in.
            # Using 'with' ensures the file is closed as soon as the block is left - even if an exception 
            # happens inside that block, so it should always be preferred for any real application.
            # source: https://stackoverflow.com/questions/39447362/equivalent-ways-to-json-load-a-file-in-python
            
        # json.load() : This method is used to parse JSON from URL or file.
        # json.loads(): This method is used to parse string with JSON content.
        # check: https://www.pythonpip.com/python-tutorials/how-to-load-json-file-using-python/#:~:text=The%20json.load%20%28%29%20is%20used%20to%20read%20the,and%20alter%20data%20in%20our%20application%20or%20system.   
        dataset = json_normalize(json_file, record_path = json_record_path, sep = json_field_separator, meta = json_metadata_preffix_list)
    
    else:
        # If it is not neither a csv nor a txt file, let's assume it is one of different
        # possible Excel files.
        print("Excel file inferred. If an error message is shown, check if a valid file extension was used: \'xlsx\', \'xls\', etc.")
            
        if (sheet_to_load is not None):        
        #Case where the user specifies which sheet of the Excel file should be loaded.
            
            if (has_header == True):
                
                dataset = pd.read_excel(file_path, sheet_name = sheet_to_load)
            
            else:
                #No header
                dataset = pd.read_excel(file_path, sheet_name = sheet_to_load, header = None)
        
        else:
            #No sheet specified
            if (has_header == True):
                
                dataset = pd.read_excel(file_path)
            
            else:
                #No header
                dataset = pd.read_excel(file_path, header = None)
    
    print(f"Dataset extracted from {file_path}. Check the 10 first rows of this dataframe:\n")
    print(dataset.head(10))
    
    return dataset   

# **Function for converting JSON object to dataframe**
- Objects may be:
    - String with JSON formatted text;
    - List with nested dictionaries (JSON formatted);
    - Dictionaries, possibly with nested dictionaries (JSON formatted).

In [ ]:
def json_obj_to_dataframe (json_obj_to_convert, json_record_path = None, json_field_separator = "_", json_metadata_preffix_list = None):
    
    import json
    import pandas as pd
    from pandas import json_normalize
    
    # json_obj_to_convert: object containing JSON, or string with JSON content to parse.
    # Objects may be: string with JSON formatted text;
    # list with nested dictionaries (JSON formatted);
    # dictionaries, possibly with nested dictionaries (JSON formatted).
    
    # https://docs.python.org/3/library/json.html
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html#pandas.json_normalize
    
    # json_record_path (string): manipulate parameter 'record_path' from json_normalize method.
    # Path in each object to list of records. If not passed, data will be assumed to 
    # be an array of records. If a given field from the JSON stores a nested JSON (or a nested
    # dictionary) declare it here to decompose the content of the nested data. e.g. if the field
    # 'books' stores a nested JSON, declare, json_record_path = 'books'
    
    # json_field_separator = "_" (string). Manipulates the parameter 'sep' from json_normalize method.
    # Nested records will generate names separated by sep. 
    # e.g., for json_field_separator = ".", {‘foo’: {‘bar’: 0}} -> foo.bar.
    # Then, if a given field 'main_field' stores a nested JSON with fields 'field1', 'field2', ...
    # the name of the columns of the dataframe will be formed by concatenating 'main_field', the
    # separator, and the names of the nested fields: 'main_field_field1', 'main_field_field2',...
    
    # json_metadata_preffix_list: list of strings (in quotes). Manipulates the parameter 
    # 'meta' from json_normalize method. Fields to use as metadata for each record in resulting 
    # table. Declare here the non-nested fields, i.e., the fields in the principal JSON. They
    # will be repeated in the rows of the dataframe to give the metadata (context) of the rows.
    
    # e.g. Suppose a JSON with the following structure: {'name': 'Mary', 'last': 'Shelley',
    # 'books': [{'title': 'Frankestein', 'year': 1818}, {'title': 'Mathilda ', 'year': 1819},{'title': 'The Last Man', 'year': 1826}]},
    # Here, there are nested JSONs in the field 'books'. The fields that are not nested
    # are 'name' and 'last'.
    # Then, json_record_path = 'books'
    # json_metadata_preffix_list = ['name', 'last']
    
    json_file = json.loads(json_obj_to_convert)
    # json.load() : This method is used to parse JSON from URL or file.
    # json.loads(): This method is used to parse string with JSON content.
    # e.g. .json.loads() must be used to read a string with JSON and convert it to a flat file
    # like a dataframe.
    # check: https://www.pythonpip.com/python-tutorials/how-to-load-json-file-using-python/#:~:text=The%20json.load%20%28%29%20is%20used%20to%20read%20the,and%20alter%20data%20in%20our%20application%20or%20system.
    dataset = json_normalize(json_file, record_path = json_record_path, sep = json_field_separator, meta = json_metadata_preffix_list)
    
    print(f"JSON object {json_obj_to_convert} converted to a flat dataframe object. Check the 10 first rows of this dataframe:\n")
    print(dataset.head(10))
    
    return dataset

# **Function for concatenating (SQL UNION) multiple dataframes**
- Vertical concatenation of the dataframes.
- Equivalent to SQL Union: vertical stack/append of the tables.

In [50]:
def UNION_DATAFRAMES (list_of_dataframes, what_to_append = 'rows', ignore_index_on_union = True, sort_values_on_union = True, union_join_type = None):
    
    import pandas as pd
    #JOIN can be 'inner' to perform an inner join, eliminating the missing values
    #The default (None) is 'outer': the dataframes will be stacked on the columns with
    #same names but, in case there is no correspondence, the row will present a missing
    #value for the columns which are not present in one of the dataframes.
    #When using the 'inner' method, only the common columns will remain
    
    #list_of_dataframes must be a list containing the dataframe objects
    # example: list_of_dataframes = [df1, df2, df3, df4]
    #Notice that the dataframes are objects, not strings. Therefore, they should not
    # be declared inside quotes.
    # There is no limit of dataframes. In this example, we will concatenate 4 dataframes.
    # If list_of_dataframes = [df1, df2, df3] we would concatenate 3, and if
    # list_of_dataframes = [df1, df2, df3, df4, df5] we would concatenate 5 dataframes.
    
    # what_to_append = 'rows' for appending the rows from one dataframe
    # into the other; what_to_append = 'columns' for appending the columns
    # from one dataframe into the other (horizontal or lateral append).
    
    # When what_to_append = 'rows', Pandas .concat method is defined as
    # axis = 0, i.e., the operation occurs in the row level, so the rows
    # of the second dataframe are added to the bottom of the first one.
    # It is the SQL union, and creates a dataframe with more rows, and
    # total of columns equals to the total of columns of the first dataframe
    # plus the columns of the second one that were not in the first dataframe.
    # When what_to_append = 'columns', Pandas .concat method is defined as
    # axis = 1, i.e., the operation occurs in the column level: the two
    # dataframes are laterally merged using the index as the key, 
    # preserving all columns from both dataframes. Therefore, the number of
    # rows will be the total of rows of the dataframe with more entries,
    # and the total of columns will be the sum of the total of columns of
    # the first dataframe with the total of columns of the second dataframe.
    
    #The other parameters are the same from Pandas .concat method.
    # ignore_index_on_union = ignore_index;
    # sort_values_on_union = sort
    # union_join_type = join
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html
    
    #Check Datacamp course Joining Data with pandas, Chap.3, 
    # Advanced Merging and Concatenating
    
    # Check axis:
    if (what_to_append == 'rows'):
        
        AXIS = 0
    
    elif (what_to_append == 'columns'):
        
        AXIS = 1
    
    else:
        print("No valid string was input to what_to_append, so appending rows (vertical append, equivalent to SQL UNION).")
        AXIS = 0
    
    if (union_join_type == 'inner'):
        
        print("Warning: concatenating dataframes using the \'inner\' join method, that removes missing values.")
        concat_df = pd.concat(list_of_dataframes, axis = AXIS, ignore_index = ignore_index_on_union, sort = sort_values_on_union, join = union_join_type)
    
    else:
        
        #In case None or an invalid value is provided, use the default 'outer', by simply
        # not declaring the 'join':
        concat_df = pd.concat(list_of_dataframes, axis = AXIS, ignore_index = ignore_index_on_union, sort = sort_values_on_union)
    
    # Pandas .head(Y) method results in a dataframe containing the first Y rows of the 
    # original dataframe. The default .head() is Y = 5. Print first 10 rows of the 
    # new dataframe:
    print("Dataframes successfully concatenated. Check the 10 first rows of new dataframe:\n")
    print(concat_df.head(10))
    
    #Now return the concatenated dataframe:
    
    return concat_df

# **Function for column filtering (selecting); or column renaming**

In [ ]:
def col_filter_rename (df, cols_list, mode = 'filter'):
    
    import pandas as pd
    
    #mode = 'filter' for filtering only the list of columns passed as cols_list;
    #mode = 'rename' for renaming the columns with the names passed as cols_list.
    
    #cols_list = list of strings containing the names (headers) of the columns to select
    # (filter); or to be set as the new columns' names, according to the selected mode.
    # For instance: cols_list = ['col1', 'col2', 'col3'] will 
    # select columns 'col1', 'col2', and 'col3' (or rename the columns with these names). 
    # Declare the names inside quotes.
    
    print(f"Original columns in the dataframe:\n{df.columns}")
    
    if (mode == 'filter'):
        
        #filter the dataframe so that it will contain only the cols_list.
        df = df[cols_list]
        print("Dataframe filtered according to the list provided.")
        
    elif (mode == 'rename'):
        
        #Check if the number of columns of the dataset is equal to the number of elements
        # of the new list. It will avoid raising an exception error.
        boolean_filter = (len(cols_list) == len(df.columns))
        
        if (boolean_filter == False):
            #Impossible to rename, number of elements are different.
            print("The number of columns of the dataframe is different from the number of elements of the list. Please, provide a list with number of elements equals to the number of columns.")
        
        else:
            #Same number of elements, so that we can update the columns' names.
            df.columns = cols_list
            print("Dataframe columns renamed according to the list provided.")
            print("Warning: the substitution is element-wise: the first element of the list is now the name of the first column, and so on, ..., so that the last element is the name of the last column.")
        
        
    else:
        print("Enter a valid mode: \'filter\' or \'rename\'.")
    
    return df

# **Function for plotting the bar chart**
- Bars may be vertically or horizontally oriented.
- Bar charts are plotted after selecting an aggregation function, and the cumulative percent curve may be obtained and plotted with the bars (in secondary axis).
- To obtain a **Pareto chart**, keep `aggregate_function = 'sum'`, `plot_cumulative_percent = True`, and `orientation = 'vertical'`.

In [5]:
def bar_chart (df, categorical_var_name, response_var_name, aggregate_function = 'sum', add_suffix_to_aggregated_col = True, suffix = None, calculate_and_plot_cumulative_percent = True, orientation = 'vertical', limit_of_plotted_categories = None, horizontal_axis_title = None, vertical_axis_title = None, plot_title = None, x_axis_rotation = 70, y_axis_rotation = 0, grid = True, export_png = False, directory_to_save = None, file_name = None, png_resolution_dpi = 110):

    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    
    # df: dataframe being analyzed
    
    # categorical_var_name: string (inside quotes) containing the name 
    # of the column to be analyzed. e.g. 
    # categorical_var_name = "column1"
    
    # response_var_name: string (inside quotes) containing the name 
    # of the column that stores the response correspondent to the
    # categories. e.g. response_var_name = "response_feature" 
    
    # aggregate_function = 'sum': String defining the aggregation 
    # method that will be applied. Possible values:
    # 'median', 'mean', 'mode', 'sum', 'min', 'max', 'variance',
    # 'standard_deviation','10_percent_quantile', '20_percent_quantile',
    # '25_percent_quantile', '30_percent_quantile', '40_percent_quantile',
    # '50_percent_quantile', '60_percent_quantile', '70_percent_quantile',
    # '75_percent_quantile', '80_percent_quantile', '90_percent_quantile',
    # and '95_percent_quantile'.
    # To use another aggregate function, the method must be added to the
    # dictionary of methods agg_methods_dict, defined in the function.
    # If None or an invalid function is input, 'sum' will be used.
    
    # add_suffix_to_aggregated_col = True will add a suffix to the
    # aggregated column. e.g. 'responseVar_mean'. If add_suffix_to_aggregated_col 
    # = False, the aggregated column will have the original column name.
    
    # suffix = None. Keep it None if no suffix should be added, or if
    # the name of the aggregate function should be used as suffix, after
    # "_". Alternatively, set it as a string. As recommendation, put the
    # "_" sign in the beginning of this string to separate the suffix from
    # the original column name. e.g. if the response variable is 'Y' and
    # suffix = '_agg', the new aggregated column will be named as 'Y_agg'
    
    # calculate_and_plot_cumulative_percent = True to calculate and plot
    # the line of cumulative percent, or 
    # calculate_and_plot_cumulative_percent = False to omit it.
    # This feature is only shown when aggregate_function = 'sum', 'median',
    # 'mean', or 'mode'. So, it will be automatically set as False if 
    # another aggregate is selected.
    
    # orientation = 'vertical' is the standard, and plots vertical bars
    # (perpendicular to the X axis). In this case, the categories are shown
    # in the X axis, and the correspondent responses are in Y axis.
    # Alternatively, orientation = 'horizontal' results in horizontal bars.
    # In this case, categories are in Y axis, and responses in X axis.
    # If None or invalid values are provided, orientation is set as 'vertical'.
    
    # Note: to obtain a Pareto chart, keep aggregate_function = 'sum',
    # plot_cumulative_percent = True, and orientation = 'vertical'.
    
    # limit_of_plotted_categories: integer value that represents
    # the maximum of categories that will be plot. Keep it None to plot
    # all categories. Alternatively, set an integer value. e.g.: if
    # limit_of_plotted_categories = 4, but there are more categories,
    # the dataset will be sorted in descending order and: 1) The remaining
    # categories will be sum in a new category named 'others' if the
    # aggregate function is 'sum'; 2) Or the other categories will be simply
    # omitted from the plot, for other aggregate functions. Notice that
    # it limits only the variables in the plot: all of them will be
    # returned in the dataframe.
    # Use this parameter to obtain a cleaner plot. Notice that the remaining
    # columns will be aggregated as 'others' even if there is a single column
    # beyond the limit.
    
    
    # Create a local copy of the dataframe to manipulate:
    
    DATASET = df
    
    # Create the dictionary of possible aggregates, to define the
    # aggregation method, according to the set by the user:
    agg_methods_dict = {
        
        'median': DATASET.groupby(categorical_var_name)[response_var_name].median(),
        'mean': DATASET.groupby(categorical_var_name)[response_var_name].mean(),
        'mode': DATASET.groupby(categorical_var_name)[response_var_name].mode(),
        'sum': DATASET.groupby(categorical_var_name)[response_var_name].sum(),
        'min': DATASET.groupby(categorical_var_name)[response_var_name].min(),
        'max': DATASET.groupby(categorical_var_name)[response_var_name].max(),
        'variance': DATASET.groupby(categorical_var_name)[response_var_name].var(),
        'standard_deviation': DATASET.groupby(categorical_var_name)[response_var_name].std(),
        '10_percent_quantile': DATASET.groupby(categorical_var_name)[response_var_name].quantile(0.10),
        '20_percent_quantile': DATASET.groupby(categorical_var_name)[response_var_name].quantile(0.20),
        '25_percent_quantile': DATASET.groupby(categorical_var_name)[response_var_name].quantile(0.25),
        '30_percent_quantile': DATASET.groupby(categorical_var_name)[response_var_name].quantile(0.30),
        '40_percent_quantile': DATASET.groupby(categorical_var_name)[response_var_name].quantile(0.40),
        '50_percent_quantile': DATASET.groupby(categorical_var_name)[response_var_name].quantile(0.50),
        '60_percent_quantile': DATASET.groupby(categorical_var_name)[response_var_name].quantile(0.60),
        '70_percent_quantile': DATASET.groupby(categorical_var_name)[response_var_name].quantile(0.70),
        '75_percent_quantile': DATASET.groupby(categorical_var_name)[response_var_name].quantile(0.75),
        '80_percent_quantile': DATASET.groupby(categorical_var_name)[response_var_name].quantile(0.80),
        '90_percent_quantile': DATASET.groupby(categorical_var_name)[response_var_name].quantile(0.90),
        '95_percent_quantile': DATASET.groupby(categorical_var_name)[response_var_name].quantile(0.95)
    }
    
    # check if the function was not set in the dictionary. If not,
    # use 'sum'
    if (aggregate_function not in (agg_methods_dict.keys())):
        
        aggregate_function = 'sum'
        print("Invalid or no aggregation function input, so using the default \'sum\'.")
    
    # Select the method in the dictionary and apply it. To access a value
    # 'val' correspondent to the key 'key' from a dictionary dict, we
    # declare: dict['key'], just as accessing a column from a dataframe.
    
    # The value will be the application of the method itself, i.e., the
    # dataset will be aggregated:
    DATASET = agg_methods_dict[aggregate_function]
    
    # If an aggregate function different from 'sum', 'mean', 'median' or 'mode' 
    # is used with plot_cumulative_percent = True, 
    # set plot_cumulative_percent = False:
    # (check if aggregate function is not in the list of allowed values):
    if ((aggregate_function not in ['sum', 'mean', 'median', 'mode']) & (calculate_and_plot_cumulative_percent == True)):
        
        calculate_and_plot_cumulative_percent = False
        print("The cumulative percent is only calculated when aggregate_function = \'sum\', \'mean\', \'median\', or \'mode\'. So, plot_cumulative_percent was set as False.")
    
    # Guarantee that the columns from the aggregated dataset have the correct
    
    # Let's create a list of the new column names
    # The first element is categorical_var_name, which is not modified:
    list_of_cols = [categorical_var_name]
    
    # Check if add_suffix_to_aggregated_col is False. If it is, simply
    # repeat the original response_var_name:
    if (add_suffix_to_aggregated_col == False):
        
        list_of_cols.append(response_var_name)
    
    else:
        # Let's add a suffix. Check if suffix is None. If it is,
        # set "_" + aggregate_function as suffix:
        
        if (suffix is None):
            suffix = "_" + aggregate_function
        
        # Now, append response_var_name + suffix to the list to
        # create the name of the new aggregated column:
        response_var_name = response_var_name + suffix
        list_of_cols.append(response_var_name)
    
    # Now, rename the columns of the aggregated dataset as the list
    # list_of_cols:
    DATASET.columns = list_of_cols
    
    # Let's sort the dataframe.
    
    # Order the dataframe in descending order by the response.
    # If there are equal responses, order them by category, in
    # ascending order; put the missing values in the first position
    # To pass multiple columns and multiple types of ordering, we use
    # lists. If there was a single column to order by, we would declare
    # it as a string. If only one order of ascending was used, we would
    # declare it as a simple boolean
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html
    
    DATASET = DATASET.sort_values(by = [response_var_name, categorical_var_name], ascending = [False, True], na_position = 'first')
    
    # Now, reset index positions:
    DATASET = DATASET.reset_index(drop = True)
    
    # plot_cumulative_percent = True, create a column to store the
    # cumulative percent:
    if (calculate_and_plot_cumulative_percent): 
        # Run the following code if the boolean value is True (implicity)
        
        # Calculate the total sum of the array correspondent to
        # the column (series) response_var_name
        total_sum = np.sum(np.array(DATASET[response_var_name]))
        
        # Create a column series for the cumulative sum:
        cumsum_col = response_var_name + "_cumsum"
        DATASET[cumsum_col] = DATASET[response_var_name].cumsum()
        
        # Now, create a column for the accumulated percent
        # by dividing cumsum_col by total_sum and multiplying it by
        # 100 (%):
        cum_pct_col = response_var_name + "_cum_pct"
        DATASET[cum_pct_col] = (DATASET[cumsum_col])/(total_sum)*100
        print(f"Successfully calculated cumulative sum and cumulative percent correspondent to the response variable {response_var_name}.")
    
    print("Successfully aggregated and ordered the dataset to plot. Check the 10 first rows of this returned dataset:\n")
    print(DATASET.head(10))
    
    # Check if the total of plotted categories is limited:
    if not (limit_of_plotted_categories is None):
        
        # Since the value is not None, we have to limit it
        # Check if the limit is lower than or equal to the length of the dataframe.
        # If it is, we simply copy the columns to the series (there is no need of
        # a memory-consuming loop or of applying the head method to a local copy
        # of the dataframe):
        df_length = len(DATASET)
            
        if (limit_of_plotted_categories <= df_length):
            # Simply copy the columns to the graphic series:
            categories = DATASET[categorical_var_name]
            responses = DATASET[response_var_name]
            # If there is a cum_pct column, copy it to a series too:
            if (calculate_and_plot_cumulative_percent):
                cum_pct = plotted_df[cum_pct_col]
        
        else:
            # The limit is lower than the total of categories,
            # so we actually have to limit the size of plotted df:
        
            # If aggregate_function is not 'sum', we simply apply
            # the head method to obtain the first rows (number of
            # rows input as parameter; if no parameter is input, the
            # number of 5 rows is used):
            if (aggregate_function != 'sum'):
                # Limit to the number limit_of_plotted_categories:
                # create another local copy of the dataframe not to
                # modify the returned dataframe object:
                plotted_df = DATASET.head(limit_of_plotted_categories)

                # Create the series of elements to plot:
                categories = plotted_df[categorical_var_name]
                responses = plotted_df[response_var_name]
                # If the cumulative percent was obtained, create the series for it:
                if (calculate_and_plot_cumulative_percent):
                    cum_pct = plotted_df[cum_pct_col]

            else:

                # Firstly, copy the elements that will be kept to x, y and (possibly) cum_pct
                # lists.
                # Start the lists:
                categories = []
                responses = []
                if (calculate_and_plot_cumulative_percent):
                    cum_pct = [] # start this list only if its needed to save memory

                for i in range (0, limit_of_plotted_categories):
                    # i goes from 0 (first index) to limit_of_plotted_categories - 1
                    # (index of the last category to be kept):
                    # copy the elements from the DATASET to the list
                    # category is the 1st column (column 0); response is the 2nd (col 1);
                    # and cumulative percent is the 4th (col 3):
                    categories.append(DATASET.iloc[i, 0])
                    responses.append(DATASET.iloc[i, 1])
                    
                    if (calculate_and_plot_cumulative_percent):
                        cum_pct.append(DATASET.iloc[i, 3]) # only if there is something to iloc
                    
                # Now, i = limit_of_plotted_categories - 1
                # Create a variable to store the sum of other responses
                other_responses = 0
                # loop from i = limit_of_plotted_categories to i = df_length-1, index
                # of the last element. Notice that this loop may have a single call, if there
                # is only one element above the limit:
                for i in range (limit_of_plotted_categories, (df_length - 1)):
                    
                    other_responses = other_responses + (DATASET.iloc[i, 1])
                
                # Now, add the last elements to the series:
                # The last category is named 'others':
                categories.append('others')
                # The correspondent aggregated response is the value 
                # stored in other_responses:
                responses.append(other_responses)
                # The cumulative percent is 100%, since this must be the sum of all
                # elements (the previous ones plus the ones aggregated as 'others'
                # must totalize 100%).
                # On the other hand, the cumulative percent is stored only if needed:
                cum_pct.append(100)
    
    else:
        # This is the situation where there is no limit of plotted categories. So, we
        # simply copy the columns to the plotted series (it is equivalent to the 
        # situation where there is a limit, but the limit is equal or inferior to the
        # size of the dataframe):
        categories = DATASET[categorical_var_name]
        responses = DATASET[response_var_name]
        # If there is a cum_pct column, copy it to a series too:
        if (calculate_and_plot_cumulative_percent):
            cum_pct = plotted_df[cum_pct_col]
    
    
    # Now the data is prepared and we only have to plot 
    # categories, responses, and cum_pct:
    
    # Set labels and titles for the case they are None
    if (plot_title is None):
        plot_title = f"Bar_chart_for_{response_var_name}_by_{categorical_var_name}"
    
    if (horizontal_axis_title is None):

        horizontal_axis_title = categorical_var_name

    if (vertical_axis_title is None):
        # Notice that response_var_name already has the suffix indicating the
        # aggregation function
        vertical_axis_title = response_var_name
    
    fig, ax1 = plt.subplots()
    
    #ROTATE X AXIS IN XX DEGREES
    plt.xticks(rotation = x_axis_rotation)
    # XX = 70 DEGREES x_axis (Default)
    #ROTATE Y AXIS IN XX DEGREES:
    plt.yticks(rotation = y_axis_rotation)
    # XX = 0 DEGREES y_axis (Default)
    
    plt.title(plot_title)
    ax1.set_xlabel(horizontal_axis_title)
    ax1.set_ylabel(vertical_axis_title, color = 'blue')
    
    if (orientation == 'horizontal'):
        
        # Horizontal bars used - barh method (bar horizontal):
        # https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.barh.html
        # Now, the categorical variables stored in series categories must be
        # positioned as the vertical axis Y, whereas the correspondent responses
        # must be in the horizontal axis X.
        ax1.barh(categories, responses, color = 'blue', label = categorical_var_name)
        #.barh(y, x, ...)
        
        if (calculate_and_plot_cumulative_percent):
            # Let's plot the line for the cumulative percent
            # Set the grid for the bar chart as False. If it is True, there will
            # be to grids, one for the bars and other for the percents, making 
            # the image difficult to interpretate:
            ax1.grid(False)
            
            # Create the twin plot for the cumulative percent:
            ax2 = ax1.twinx()
            # Here, the x axis must be the cum_pct value, and the Y
            # axis must be categories (it must be correspondent to the
            # bar chart)
            ax2.plot(cum_pct, categories, '-ro', color = 'red', label = "cumulative\npercent")
            #.plot(x, y, ...)
            ax2.tick_params('x', color = 'red')
            ax2.set_ylabel("Cumulative Percent (\%)", color = 'red')
            ax2.legend()
            ax2.grid(grid) # shown if user set grid = True
            # If user wants to see the grid, it is shown only for the cumulative line.
        
        else:
            # There is no cumulative line, so the parameter grid must control 
            # the bar chart's grid
            ax1.legend()
            ax1.grid(grid)
        
    else: 
        # If None or an invalid orientation was used, set it as vertical
        # Use Matplotlib standard bar method (vertical bar):
        # https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.bar.html#matplotlib.pyplot.bar
        
        # In this standard case, the categorical variables (categories) are positioned
        # as X, and the responses as Y:
        ax1.bar(categories, responses, color = 'blue', label = categorical_var_name)
        #.bar(x, y, ...)
        
        if (calculate_and_plot_cumulative_percent):
            # Let's plot the line for the cumulative percent
            # Set the grid for the bar chart as False. If it is True, there will
            # be to grids, one for the bars and other for the percents, making 
            # the image difficult to interpretate:
            ax1.grid(False)
            
            # Create the twin plot for the cumulative percent:
            ax2 = ax1.twinx()
            ax2.plot(categories, cum_pct, '-ro', color = 'red', label = "cumulative\npercent")
            #.plot(x, y, ...)
            ax2.tick_params('y', color = 'red')
            ax2.set_ylabel("Cumulative Percent (\%)", color = 'red')
            ax2.legend()
            ax2.grid(grid) # shown if user set grid = True
            # If user wants to see the grid, it is shown only for the cumulative line.
        
        else:
            # There is no cumulative line, so the parameter grid must control 
            # the bar chart's grid
            ax1.legend()
            ax1.grid(grid)
    
    # Notice that the .plot method is used for generating the plot for both orientations.
    # It is different from .bar and .barh, which specify the orientation of a bar; or
    # .hline (creation of an horizontal constant line); or .vline (creation of a vertical
    # constant line).
    
    # Now the parameters specific to the configurations are finished, so we can go back
    # to the general code:
    
    if (export_png == True):
        # Image will be exported
        import os
        
        #check if the user defined a directory path. If not, set as the default root path:
        if (directory_to_save is None):
            #set as the default
            directory_to_save = "/"
        
        #check if the user defined a file name. If not, set as the default name for this
        # function.
        if (file_name is None):
            #set as the default
            file_name = "bar_chart"
        
        #check if the user defined an image resolution. If not, set as the default 110 dpi
        # resolution.
        if (png_resolution_dpi is None):
            #set as 110 dpi
            png_resolution_dpi = 110
        
        #Get the new_file_path
        new_file_path = os.path.join(directory_to_save, file_name)
        
        #Export the file to this new path:
        # The extension will be automatically added by the savefig method:
        plt.savefig(new_file_path, dpi = png_resolution_dpi, quality = 100, format = 'png', transparent = False) 
        #quality could be set from 1 to 100, where 100 is the best quality
        #format (str, supported formats) = 'png', 'pdf', 'ps', 'eps' or 'svg'
        #transparent = True or False
        # For other parameters of .savefig method, check https://indianaiproduction.com/matplotlib-savefig/
        print (f"Figure exported as \'{new_file_path}.png\'. Any previous file in this root path was overwritten.")
    
    #Set image size (x-pixels, y-pixels) for printing in the notebook's cell:
    plt.figure(figsize=(12, 8))
    #fig.tight_layout()
    
    ## Show an image read from an image file:
    ## import matplotlib.image as pltimg
    ## img=pltimg.imread('mydecisiontree.png')
    ## imgplot = plt.imshow(img)
    ## See linkedIn Learning course: "Supervised machine learning and the technology boom",
    ##  Ex_Files_Supervised_Learning, Exercise Files, lesson '03. Decision Trees', '03_05', 
    ##  '03_05_END.ipynb'
    plt.show()
    
    return DATASET

# **Function for splitting the dataframe into train and test subsets**

In [ ]:
def split_data_into_train_and_test (X, y, percent_of_data_used_for_model_training = 75):
    
    import numpy as np
    import pandas as pd
    from sklearn.model_selection import train_test_split
    
    # X = subset of predictive variables (dataframe).
    # y = subset of response variable (series).
    
    # percent_of_data_used_for_model_training: float from 0 to 100,
    # representing the percent of data used for training the model
    
    # Convert the percent to fraction.
    train_fraction = (percent_of_data_used_for_model_training / 100)
    # Calculate the test fraction:
    test_fraction = (1 - train_fraction)
    
    #Funcao para dividir os dados (split em treino e teste)
    X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size = test_fraction, random_state = 0)
    #test_size: proportion: 0.25 used for test
    #test_size = 0.25 = 25% of data used for tests 
    #-> then, 0.75 = 75% of data used for training the Machine Learning model
    
    print(f"X and y successfully splitted into train: X_train, y_train ({percent_of_data_used_for_model_training}\% of data); and test subsets: X_test, y_test ({100 - percent_of_data_used_for_model_training}\% of data).")
    # the slash is used to pass a prohibited character % to the string: it is ignorated, so it is printed.
    
    return X_train, X_test, y_train, y_test

# **Function for Ordinary Least Squares (OLS) Linear Regression**
    - This function runs the 'bar_chart' function. Certify that this function was properly loaded.
- Fits a linear model with coefficients w = (w1, …, wp) to minimize the residual sum of squares between the observed targets in the dataset, and the targets predicted by the linear approximation.

In [ ]:
def ols_linear_reg (X_train, y_train):
    
    # check Scikit-learn documentation: 
    # https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html?msclkid=636b4046c01b11ec973dee34641f67b0
    # This function runs the 'bar_chart' function. Certify that this function was properly loaded.
    
    import numpy as np
    import pandas as pd
    from sklearn.linear_model import LinearRegression
    
    # X_train = subset of predictive variables (dataframe).
    # y_train = subset of response variable (series).
    
    # Create an instance (object) from the class LinearRegression:
    ols_linear_reg_model = LinearRegression()
    
    # Fit the model:
    ols_linear_reg_model = ols_linear_reg_model.fit(X_train, y_train)
    
    # Set the list of the predictors:
    # Use the list attribute to guarantee that it is a list:
    predictive_features = list(X_train.columns)
    # Append the 'intercept' to this list:
    predictive_features.append('intercept')
    
    # Get the list of coefficients. Apply the list method to convert the
    # array from .coef_ to a list:
    reg_coefficients = list(ols_linear_reg_model.coef_)
    
    # Append the intercept coefficient to this list:
    reg_coefficients.append(ols_linear_reg_model.intercept_)
    
    # Create the regression dictionary:
    reg_dict = {'predictive_features': predictive_features,
               'regression_coefficients': reg_coefficients}
    
    # Convert it to a Pandas dataframe:
    ols_feature_importance_df = pd.DataFrame(data = reg_dict)
    
    # Now sort the dataframe in descending order of coefficient, and ascending order of
    # feature (when sorting by multiple columns, we pass a list of columns to by and a 
    # list of booleans to ascending, instead of passing a simple string to by and a boolean
    # to ascending. The element on a given index from the list by corresponds to the boolean
    # with the same index in ascending):
    ols_feature_importance_df = ols_feature_importance_df.sort_values(by = ['regression_coefficients', 'predictive_features'], ascending = [False, True])
    
    # Now that the dataframe is sorted in descending order, it represents the feature
    # importance ranking.
    
    # Restart the indices:
    ols_feature_importance_df = ols_feature_importance_df.reset_index(drop = True)
    
    print("Successfully obtained the linear regression.")
    print(f"R² = {ols_linear_reg_model.score(X_train, y_train)}\n")
    print("Check the parameters of the estimator:")
    print(ols_linear_reg_model.get_params(deep = True))
    print("Returning the model object \'ols_linear_reg_model\' and the dataframe \'ols_feature_importance_df\' with the feature importance ranking (regression coefficients in descending order). Check the ranking below (first 20 features):\n")
    print(ols_feature_importance_df.head(20))
    
    print("\n") #line break
    print("To predict the model output y_pred for a dataframe X, declare: y_pred = ols_linear_reg_model.predict(X)\n")
    print("For a one-dimensional correlation, the one-dimension array or list with format X_train = [x1, x2, ...] must be converted into a dataframe subset, X_train = [[x1, x2, ...]] before the prediction. To do so, create a list with X_train as its element: X_train = [X_train], or use the numpy.reshape(-1,1):")
    print("X_train = np.reshape(np.array(X_train), (-1, 1))")
    # numpy reshape: https://numpy.org/doc/1.21/reference/generated/numpy.reshape.html?msclkid=5de33f8bc02c11ec803224a6bd588362
    
    print("\n")
    print("Check the feature importance bar chart:\n")
    
    # Obtain the bar chart. Set the local variables for using as bar_chart function parameters:
    DATASET = ols_feature_importance_df
    CATEGORICAL_VAR_NAME = 'predictive_features'
    RESPONSE_VAR_NAME = 'regression_coefficients'
    AGGREGATE_FUNCTION = 'sum'
    ADD_SUFFIX_TO_AGGREGATED_COL = True
    SUFFIX = None
    CALCULATE_AND_PLOT_CUMULATIVE_PERCENT = False
    ORIENTATION = 'vertical'
    X_AXIS_ROTATION = 70
    Y_AXIS_ROTATION = 0
    GRID = True
    HORIZONTAL_AXIS_TITLE = 'Feature'
    VERTICAL_AXIS_TITLE = 'Regression_coefficients'
    PLOT_TITLE = 'Feature_ranking'
    EXPORT_PNG = False
    DIRECTORY_TO_SAVE = None
    FILE_NAME = None
    PNG_RESOLUTION_DPI = 110
    
    # use underscore to ignore the dataframe, and simply obtain the plot:
    _ = bar_chart (df = DATASET, categorical_var_name = CATEGORICAL_VAR_NAME, response_var_name = RESPONSE_VAR_NAME, aggregate_function = AGGREGATE_FUNCTION, add_suffix_to_aggregated_col = ADD_SUFFIX_TO_AGGREGATED_COL, suffix = SUFFIX, calculate_and_plot_cumulative_percent = CALCULATE_AND_PLOT_CUMULATIVE_PERCENT, orientation = ORIENTATION, limit_of_plotted_categories = LIMIT_OF_PLOTTED_CATEGORIES, horizontal_axis_title = HORIZONTAL_AXIS_TITLE, vertical_axis_title = VERTICAL_AXIS_TITLE, plot_title = PLOT_TITLE, x_axis_rotation = X_AXIS_ROTATION, y_axis_rotation = Y_AXIS_ROTATION, grid = GRID, export_png = EXPORT_PNG, directory_to_save = DIRECTORY_TO_SAVE, file_name = FILE_NAME, png_resolution_dpi = PNG_RESOLUTION_DPI)

    return ols_linear_reg_model, ols_feature_importance_df

# **Function for Ridge Linear Regression**
    - This function runs the 'bar_chart' function. Certify that this function was properly loaded.
- Linear least squares with l2 regularization.
- Minimizes the objective function: `||y - Xw||^2_2 + alpha * ||w||^2_2`
- This model solves a regression model where the loss function is the linear least squares function and regularization is given by the l2-norm. 
- Also known as Ridge Regression or Tikhonov regularization.
- This estimator has built-in support for multi-variate regression (i.e., when y is a 2d-array of shape (n_samples, n_targets)).

In [ ]:
def ridge_linear_reg (X_train, y_train, alpha_hyperparameter = 1.0, maximum_of_allowed_iterations = 20000):
    
    # check Scikit-learn documentation: 
    # https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge
    # This function runs the 'bar_chart' function. Certify that this function was properly loaded.
    
    import numpy as np
    import pandas as pd
    from sklearn.linear_model import Ridge
    
    # X_train = subset of predictive variables (dataframe).
    # y_train = subset of response variable (series).
    
    # hyperparameters: alpha = ALPHA_HYPERPARAMETER and MAXIMUM_OF_ALLOWED_ITERATIONS = max_iter

    # MAXIMUM_OF_ALLOWED_ITERATIONS = integer representing the maximum number of iterations
    # that the optimization algorithm can perform. Depending on data, convergence may not be
    # reached within this limit, so you may need to increase this hyperparameter.

    # alpha is the regularization strength and must be a positive float value. 
    # Regularization improves the conditioning of the problem and reduces the variance 
    # of the estimates. Larger values specify stronger regularization.
    
    # alpha = 0 is equivalent to an ordinary least square, solved by the LinearRegression 
    # object. For numerical reasons, using alpha = 0 is not advised. 
    # Given this, you should use the ols_linear_reg function instead.
    
    # Create an instance (object) from the class Ridge:
    ridge_linear_reg_model = Ridge(alpha = alpha_hyperparameter, max_iter = maximum_of_allowed_iterations)
    
    # Fit the model:
    ridge_linear_reg_model = ridge_linear_reg_model.fit(X_train, y_train)
    
    # Set the list of the predictors:
    # Use the list attribute to guarantee that it is a list:
    predictive_features = list(X_train.columns)
    # Append the 'intercept' to this list:
    predictive_features.append('intercept')
    
    # Get the list of coefficients. Apply the list method to convert the
    # array from .coef_ to a list:
    reg_coefficients = list(ridge_linear_reg_model.coef_)
    
    # Append the intercept coefficient to this list:
    reg_coefficients.append(ridge_linear_reg_model.intercept_)
    
    # Create the regression dictionary:
    reg_dict = {'predictive_features': predictive_features,
               'regression_coefficients': reg_coefficients}
    
    # Convert it to a Pandas dataframe:
    ridge_feature_importance_df = pd.DataFrame(data = reg_dict)
    
    # Now sort the dataframe in descending order of coefficient, and ascending order of
    # feature (when sorting by multiple columns, we pass a list of columns to by and a 
    # list of booleans to ascending, instead of passing a simple string to by and a boolean
    # to ascending. The element on a given index from the list by corresponds to the boolean
    # with the same index in ascending):
    ridge_feature_importance_df = ridge_feature_importance_df.sort_values(by = ['regression_coefficients', 'predictive_features'], ascending = [False, True])
    
    # Now that the dataframe is sorted in descending order, it represents the feature
    # importance ranking.
    
    # Restart the indices:
    ridge_feature_importance_df = ridge_feature_importance_df.reset_index(drop = True)
    
    print("Successfully obtained the linear regression.")
    print(f"R² = {ridge_linear_reg_model.score(X_train, y_train)}\n")
    print(f"Total of iterations to fit the model = {ridge_linear_reg_model.n_iter_}")
    
    if (ridge_linear_reg_model.n_iter_ == maximum_of_allowed_iterations):
        print("Warning! Total of iterations equals to the maximum allowed. It indicates that the convergence was not reached yet. Try to increase the maximum number of allowed iterations.")
    
    print("Check the parameters of the estimator:")
    print(ridge_linear_reg_model.get_params(deep = True))
    print("Returning the model object \'ridge_linear_reg_model\' and the dataframe \'ridge_feature_importance_df\' with the feature importance ranking (regression coefficients in descending order). Check the ranking below (first 20 features):\n")
    print(ridge_feature_importance_df.head(20))
    
    print("\n") #line break
    print("To predict the model output y_pred for a dataframe X, declare: y_pred = ridge_linear_reg_model.predict(X)\n")
    print("For a one-dimensional correlation, the one-dimension array or list with format X_train = [x1, x2, ...] must be converted into a dataframe subset, X_train = [[x1, x2, ...]] before the prediction. To do so, create a list with X_train as its element: X_train = [X_train], or use the numpy.reshape(-1,1):")
    print("X_train = np.reshape(np.array(X_train), (-1, 1))")
    # numpy reshape: https://numpy.org/doc/1.21/reference/generated/numpy.reshape.html?msclkid=5de33f8bc02c11ec803224a6bd588362
      
    print("\n")
    print("Check the feature importance bar chart:\n")
    
    # Obtain the bar chart. Set the local variables for using as bar_chart function parameters:
    DATASET = ridge_feature_importance_df
    CATEGORICAL_VAR_NAME = 'predictive_features'
    RESPONSE_VAR_NAME = 'regression_coefficients'
    AGGREGATE_FUNCTION = 'sum'
    ADD_SUFFIX_TO_AGGREGATED_COL = True
    SUFFIX = None
    CALCULATE_AND_PLOT_CUMULATIVE_PERCENT = False
    ORIENTATION = 'vertical'
    X_AXIS_ROTATION = 70
    Y_AXIS_ROTATION = 0
    GRID = True
    HORIZONTAL_AXIS_TITLE = 'Feature'
    VERTICAL_AXIS_TITLE = 'Regression_coefficients'
    PLOT_TITLE = 'Feature_ranking'
    EXPORT_PNG = False
    DIRECTORY_TO_SAVE = None
    FILE_NAME = None
    PNG_RESOLUTION_DPI = 110
    
    # use underscore to ignore the dataframe, and simply obtain the plot:
    _ = bar_chart (df = DATASET, categorical_var_name = CATEGORICAL_VAR_NAME, response_var_name = RESPONSE_VAR_NAME, aggregate_function = AGGREGATE_FUNCTION, add_suffix_to_aggregated_col = ADD_SUFFIX_TO_AGGREGATED_COL, suffix = SUFFIX, calculate_and_plot_cumulative_percent = CALCULATE_AND_PLOT_CUMULATIVE_PERCENT, orientation = ORIENTATION, limit_of_plotted_categories = LIMIT_OF_PLOTTED_CATEGORIES, horizontal_axis_title = HORIZONTAL_AXIS_TITLE, vertical_axis_title = VERTICAL_AXIS_TITLE, plot_title = PLOT_TITLE, x_axis_rotation = X_AXIS_ROTATION, y_axis_rotation = Y_AXIS_ROTATION, grid = GRID, export_png = EXPORT_PNG, directory_to_save = DIRECTORY_TO_SAVE, file_name = FILE_NAME, png_resolution_dpi = PNG_RESOLUTION_DPI)

    return ridge_linear_reg_model, ridge_feature_importance_df

# **Function for Lasso Linear Regression**
    - This function runs the 'bar_chart' function. Certify that this function was properly loaded.
- Linear Model trained with L1 prior as regularizer (aka the Lasso).
- The optimization objective for Lasso is: `(1 / (2 * n_samples)) * ||y - Xw||^2_2 + alpha * ||w||_1`
- Technically the Lasso model is optimizing the same objective function as the Elastic Net with l1_ratio=1.0 (no L2 penalty).

In [ ]:
def lasso_linear_reg (X_train, y_train, alpha_hyperparameter = 1.0, maximum_of_allowed_iterations = 20000):
    
    # check Scikit-learn documentation: 
    # https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html#sklearn.linear_model.Lasso
    # This function runs the 'bar_chart' function. Certify that this function was properly loaded.
    
    import numpy as np
    import pandas as pd
    from sklearn.linear_model import Lasso
    
    # X_train = subset of predictive variables (dataframe).
    # y_train = subset of response variable (series).
    
    # hyperparameters: alpha = ALPHA_HYPERPARAMETER and MAXIMUM_OF_ALLOWED_ITERATIONS = max_iter

    # MAXIMUM_OF_ALLOWED_ITERATIONS = integer representing the maximum number of iterations
    # that the optimization algorithm can perform. Depending on data, convergence may not be
    # reached within this limit, so you may need to increase this hyperparameter.

    # alpha is the regularization strength and must be a positive float value. 
    # Regularization improves the conditioning of the problem and reduces the variance 
    # of the estimates. Larger values specify stronger regularization.
    
    # alpha = 0 is equivalent to an ordinary least square, solved by the LinearRegression 
    # object. For numerical reasons, using alpha = 0 is not advised. 
    # Given this, you should use the ols_linear_reg function instead.
    
    # Create an instance (object) from the class Lasso:
    lasso_linear_reg_model = Lasso(alpha = alpha_hyperparameter, max_iter = maximum_of_allowed_iterations)
    
    # Fit the model:
    lasso_linear_reg_model = lasso_linear_reg_model.fit(X_train, y_train)
    
    # Set the list of the predictors:
    # Use the list attribute to guarantee that it is a list:
    predictive_features = list(X_train.columns)
    # Append the 'intercept' to this list:
    predictive_features.append('intercept')
    
    # Get the list of coefficients. Apply the list method to convert the
    # array from .coef_ to a list:
    reg_coefficients = list(lasso_linear_reg_model.coef_)
    
    # Append the intercept coefficient to this list:
    reg_coefficients.append(lasso_linear_reg_model.intercept_)
    
    # Create the regression dictionary:
    reg_dict = {'predictive_features': predictive_features,
               'regression_coefficients': reg_coefficients}
    
    # Convert it to a Pandas dataframe:
    lasso_feature_importance_df = pd.DataFrame(data = reg_dict)
    
    # Now sort the dataframe in descending order of coefficient, and ascending order of
    # feature (when sorting by multiple columns, we pass a list of columns to by and a 
    # list of booleans to ascending, instead of passing a simple string to by and a boolean
    # to ascending. The element on a given index from the list by corresponds to the boolean
    # with the same index in ascending):
    lasso_feature_importance_df = lasso_feature_importance_df.sort_values(by = ['regression_coefficients', 'predictive_features'], ascending = [False, True])
    
    # Now that the dataframe is sorted in descending order, it represents the feature
    # importance ranking.
    
    # Restart the indices:
    lasso_feature_importance_df = lasso_feature_importance_df.reset_index(drop = True)
    
    print("Successfully obtained the linear regression.")
    print(f"R² = {lasso_linear_reg_model.score(X_train, y_train)}\n")
    print(f"Total of iterations to fit the model = {lasso_linear_reg_model.n_iter_}")
    
    if (lasso_linear_reg_model.n_iter_ == maximum_of_allowed_iterations):
        print("Warning! Total of iterations equals to the maximum allowed. It indicates that the convergence was not reached yet. Try to increase the maximum number of allowed iterations.")
    
    print("Check the parameters of the estimator:")
    print(lasso_linear_reg_model.get_params(deep = True))
    print("Returning the model object \'lasso_linear_reg_model\' and the dataframe \'lasso_feature_importance_df\' with the feature importance ranking (regression coefficients in descending order). Check the ranking below (first 20 features):\n")
    print(lasso_feature_importance_df.head(20))
    
    print("\n") #line break
    print("To predict the model output y_pred for a dataframe X, declare: y_pred = lasso_linear_reg_model.predict(X)\n")
    print("For a one-dimensional correlation, the one-dimension array or list with format X_train = [x1, x2, ...] must be converted into a dataframe subset, X_train = [[x1, x2, ...]] before the prediction. To do so, create a list with X_train as its element: X_train = [X_train], or use the numpy.reshape(-1,1):")
    print("X_train = np.reshape(np.array(X_train), (-1, 1))")
    # numpy reshape: https://numpy.org/doc/1.21/reference/generated/numpy.reshape.html?msclkid=5de33f8bc02c11ec803224a6bd588362
      
    print("\n")
    print("Check the feature importance bar chart:\n")
    
    # Obtain the bar chart. Set the local variables for using as bar_chart function parameters:
    DATASET = lasso_feature_importance_df
    CATEGORICAL_VAR_NAME = 'predictive_features'
    RESPONSE_VAR_NAME = 'regression_coefficients'
    AGGREGATE_FUNCTION = 'sum'
    ADD_SUFFIX_TO_AGGREGATED_COL = True
    SUFFIX = None
    CALCULATE_AND_PLOT_CUMULATIVE_PERCENT = False
    ORIENTATION = 'vertical'
    X_AXIS_ROTATION = 70
    Y_AXIS_ROTATION = 0
    GRID = True
    HORIZONTAL_AXIS_TITLE = 'Feature'
    VERTICAL_AXIS_TITLE = 'Regression_coefficients'
    PLOT_TITLE = 'Feature_ranking'
    EXPORT_PNG = False
    DIRECTORY_TO_SAVE = None
    FILE_NAME = None
    PNG_RESOLUTION_DPI = 110
    
    # use underscore to ignore the dataframe, and simply obtain the plot:
    _ = bar_chart (df = DATASET, categorical_var_name = CATEGORICAL_VAR_NAME, response_var_name = RESPONSE_VAR_NAME, aggregate_function = AGGREGATE_FUNCTION, add_suffix_to_aggregated_col = ADD_SUFFIX_TO_AGGREGATED_COL, suffix = SUFFIX, calculate_and_plot_cumulative_percent = CALCULATE_AND_PLOT_CUMULATIVE_PERCENT, orientation = ORIENTATION, limit_of_plotted_categories = LIMIT_OF_PLOTTED_CATEGORIES, horizontal_axis_title = HORIZONTAL_AXIS_TITLE, vertical_axis_title = VERTICAL_AXIS_TITLE, plot_title = PLOT_TITLE, x_axis_rotation = X_AXIS_ROTATION, y_axis_rotation = Y_AXIS_ROTATION, grid = GRID, export_png = EXPORT_PNG, directory_to_save = DIRECTORY_TO_SAVE, file_name = FILE_NAME, png_resolution_dpi = PNG_RESOLUTION_DPI)

    return lasso_linear_reg_model, lasso_feature_importance_df

# **Function for Elastic Net Linear Regression**
    - This function runs the 'bar_chart' function. Certify that this function was properly loaded.
- Linear Model trained with combined L1 and L2 priors as regularizer.
- Minimizes the objective function: `1 / (2 * n_samples) * ||y - Xw||^2_2 + alpha * l1_ratio * ||w||_1 + 0.5 * alpha * (1 - l1_ratio) * ||w||^2_2`
- If you are interested in controlling the L1 and L2 penalty separately, keep in mind that this is equivalent to: `a * ||w||_1 + 0.5 * b * ||w||_2^2`
- where: `alpha = a + b and l1_ratio = a / (a + b)`
- The parameter l1_ratio corresponds to alpha in the glmnet R package while alpha corresponds to the lambda parameter in glmnet. Specifically, l1_ratio = 1 is the lasso penalty. Currently, l1_ratio <= 0.01 is not reliable, unless you supply your own sequence of alpha.

In [ ]:
def elastic_net_linear_reg (X_train, y_train, alpha_hyperparameter = 1.0, l1_ratio_hyperparameter = 0.5, maximum_of_allowed_iterations = 20000):
    
    # check Scikit-learn documentation: 
    # https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html#sklearn.linear_model.ElasticNet
    # This function runs the 'bar_chart' function. Certify that this function was properly loaded.
    
    import numpy as np
    import pandas as pd
    from sklearn.linear_model import ElasticNet
    
    # X_train = subset of predictive variables (dataframe).
    # y_train = subset of response variable (series).
    
    # hyperparameters: alpha = alpha_hyperparameter; maximum_of_allowed_iterations = max_iter;
    # and l1_ratio_hyperparameter = l1_ratio

    # MAXIMUM_OF_ALLOWED_ITERATIONS = integer representing the maximum number of iterations
    # that the optimization algorithm can perform. Depending on data, convergence may not be
    # reached within this limit, so you may need to increase this hyperparameter.

    # alpha is the regularization strength and must be a positive float value. 
    # Regularization improves the conditioning of the problem and reduces the variance 
    # of the estimates. Larger values specify stronger regularization.
    
    # l1_ratio is The ElasticNet mixing parameter (float), with 0 <= l1_ratio <= 1. 
    # For l1_ratio = 0 the penalty is an L2 penalty. For l1_ratio = 1 it is an L1 penalty. 
    # For 0 < l1_ratio < 1, the penalty is a combination of L1 and L2.
    
    # alpha = 0 and l1_ratio = 0 is equivalent to an ordinary least square, solved by 
    # the LinearRegression object. For numerical reasons, using alpha = 0 and 
    # l1_ratio = 0 is not advised. Given this, you should use the ols_linear_reg function instead.
    
    # Create an instance (object) from the class ElasticNet:
    elastic_net_linear_reg_model = ElasticNet(alpha = alpha_hyperparameter, l1_ratio = l1_ratio_hyperparameter, max_iter = maximum_of_allowed_iterations)
    
    # Fit the model:
    elastic_net_linear_reg_model = elastic_net_linear_reg_model.fit(X_train, y_train)
    
    # Set the list of the predictors:
    # Use the list attribute to guarantee that it is a list:
    predictive_features = list(X_train.columns)
    # Append the 'intercept' to this list:
    predictive_features.append('intercept')
    
    # Get the list of coefficients. Apply the list method to convert the
    # array from .coef_ to a list:
    reg_coefficients = list(elastic_net_linear_reg_model.coef_)
    
    # Append the intercept coefficient to this list:
    reg_coefficients.append(elastic_net_linear_reg_model.intercept_)
    
    # Create the regression dictionary:
    reg_dict = {'predictive_features': predictive_features,
               'regression_coefficients': reg_coefficients}
    
    # Convert it to a Pandas dataframe:
    elastic_net_feature_importance_df = pd.DataFrame(data = reg_dict)
    
    # Now sort the dataframe in descending order of coefficient, and ascending order of
    # feature (when sorting by multiple columns, we pass a list of columns to by and a 
    # list of booleans to ascending, instead of passing a simple string to by and a boolean
    # to ascending. The element on a given index from the list by corresponds to the boolean
    # with the same index in ascending):
    elastic_net_feature_importance_df = elastic_net_feature_importance_df.sort_values(by = ['regression_coefficients', 'predictive_features'], ascending = [False, True])
    
    # Now that the dataframe is sorted in descending order, it represents the feature
    # importance ranking.
    
    # Restart the indices:
    elastic_net_feature_importance_df = elastic_net_feature_importance_df.reset_index(drop = True)
    
    print("Successfully obtained the linear regression.")
    print(f"R² = {elastic_net_linear_reg_model.score(X_train, y_train)}\n")
    print(f"Total of iterations to fit the model = {elastic_net_linear_reg_model.n_iter_}")
    
    if (elastic_net_linear_reg_model.n_iter_ == maximum_of_allowed_iterations):
        print("Warning! Total of iterations equals to the maximum allowed. It indicates that the convergence was not reached yet. Try to increase the maximum number of allowed iterations.")
    
    print("Check the parameters of the estimator:")
    print(elastic_net_linear_reg_model.get_params(deep = True))
    print("Returning the model object \'elastic_net_linear_reg_model\' and the dataframe \'elastic_net_feature_importance_df\' with the feature importance ranking (regression coefficients in descending order). Check the ranking below (first 20 features):\n")
    print(elastic_net_feature_importance_df.head(20))
    
    print("\n") #line break
    print("To predict the model output y_pred for a dataframe X, declare: y_pred = elastic_net_linear_reg_model.predict(X)\n")
    print("For a one-dimensional correlation, the one-dimension array or list with format X_train = [x1, x2, ...] must be converted into a dataframe subset, X_train = [[x1, x2, ...]] before the prediction. To do so, create a list with X_train as its element: X_train = [X_train], or use the numpy.reshape(-1,1):")
    print("X_train = np.reshape(np.array(X_train), (-1, 1))")
    # numpy reshape: https://numpy.org/doc/1.21/reference/generated/numpy.reshape.html?msclkid=5de33f8bc02c11ec803224a6bd588362
     
    print("\n")
    print("Check the feature importance bar chart:\n")
    
    # Obtain the bar chart. Set the local variables for using as bar_chart function parameters:
    DATASET = elastic_net_feature_importance_df
    CATEGORICAL_VAR_NAME = 'predictive_features'
    RESPONSE_VAR_NAME = 'regression_coefficients'
    AGGREGATE_FUNCTION = 'sum'
    ADD_SUFFIX_TO_AGGREGATED_COL = True
    SUFFIX = None
    CALCULATE_AND_PLOT_CUMULATIVE_PERCENT = False
    ORIENTATION = 'vertical'
    X_AXIS_ROTATION = 70
    Y_AXIS_ROTATION = 0
    GRID = True
    HORIZONTAL_AXIS_TITLE = 'Feature'
    VERTICAL_AXIS_TITLE = 'Regression_coefficients'
    PLOT_TITLE = 'Feature_ranking'
    EXPORT_PNG = False
    DIRECTORY_TO_SAVE = None
    FILE_NAME = None
    PNG_RESOLUTION_DPI = 110
    
    # use underscore to ignore the dataframe, and simply obtain the plot:
    _ = bar_chart (df = DATASET, categorical_var_name = CATEGORICAL_VAR_NAME, response_var_name = RESPONSE_VAR_NAME, aggregate_function = AGGREGATE_FUNCTION, add_suffix_to_aggregated_col = ADD_SUFFIX_TO_AGGREGATED_COL, suffix = SUFFIX, calculate_and_plot_cumulative_percent = CALCULATE_AND_PLOT_CUMULATIVE_PERCENT, orientation = ORIENTATION, limit_of_plotted_categories = LIMIT_OF_PLOTTED_CATEGORIES, horizontal_axis_title = HORIZONTAL_AXIS_TITLE, vertical_axis_title = VERTICAL_AXIS_TITLE, plot_title = PLOT_TITLE, x_axis_rotation = X_AXIS_ROTATION, y_axis_rotation = Y_AXIS_ROTATION, grid = GRID, export_png = EXPORT_PNG, directory_to_save = DIRECTORY_TO_SAVE, file_name = FILE_NAME, png_resolution_dpi = PNG_RESOLUTION_DPI)

    return elastic_net_linear_reg_model, elastic_net_feature_importance_df

# **Function for getting a general feature ranking**

In [2]:
def general_feature_ranking (dictionary_of_feature_rankings_dataframes, eliminate_non_correspondence = False, limit_of_ranked_features = None):
    
    import numpy as np
    import pandas as pd
    
    # dictionary_of_feature_rankings_dataframes
    # The key of this dictionary must be the model name or the name of the ranking.
    # This key will be used to identify the column on the new dataframe (it will be
    # used as suffix). The correspondent value must be the feature importance ranking
    # dataframe, with configuration similar to reg_dict: it must have a column
    # named, 'predictive_features', which will be used as key for merging.
    
    # For instance, for a dictionary = {'ols_linear_regression': ols_feature_df,
    # 'ridge_linear_regression': ridge_feature_df}, the columns 'regression_coefficients'
    # will be identified as: 'regression_coefficients_ols_linear_regression' and
    # 'regression_coefficients_ridge_linear_regression'. Notice that the underscore ("_")
    # is used as suffix separator.
    
    # eliminate_non_correspondence = False. Since the dataframes will be merged using an
    # "outer" join, all entries from all dataframes will be added, possibly resulting in
    # missing values (pandas "outer" is a full outer join). 
    # Then, set eliminate_non_correspondence = True to eliminate all missing values
    # (rows with entries without correspondence).
    
    # limit_of_ranked_features = None. Alternatively, set as an integer to limit the number
    # of ranked features. e.g. limit_of_ranked_features = 20 will return a dataset with only
    # 20 features. Notice that the features are sorted in accordance to their order in the
    # input dictionary. Then, the most important ranking will be the one from the first dataframe.
    
    
    # Get the list of keys from the dictionary. This will generate an array dict_keys([])
    # that cannot be referenced throgh indexing. So, use the list attribute to convert it to
    # an indexable list:
    list_of_keys = list(dictionary_of_feature_rankings_dataframes.keys())
    
    # Get the total of dataframes that will be merged. It is the length of the list_of_keys
    total_dfs = len(list_of_keys)
    
    # Get the first key. It is the first element of the list.
    suffix_left = list_of_keys[0]
    
    # Get the correspondent dataframe by accessing the dictionary value with this key:
    df_left = dictionary_of_feature_rankings_dataframes[suffix_left]
    
    # Now let's convert suffix_left to an appropriate suffix for merging.
    # Use the str attribute to guarantee that the key was properly read as a string instead
    # of other type. Then, there will be no concatenation errors:
    suffix_left = str(suffix_left)
    # Concatenate an underscore on the left of suffix_left to obtain the suffix for merging.
    suffix_left = "_" + suffix_left
    
    # Now we have the first dataframe and the first suffix. We must loop through the rest of
    # list_of_keys list starting from index i = 1 (second dataframe) to merge all with this
    # first one:
    
    for i in range (1, total_dfs):
        
        # goes from i = 1 to i = (total_dfs - 1), index of the last dictionary key.
        # Get the new dataframe to merge. It is the i-th key from list of keys:
        suffix_right = list_of_keys[i]
        
        # Access this dataframe on the dictionary:
        df_right = dictionary_of_feature_rankings_dataframes[suffix_right]
        
        # Now let's convert suffix_right to an appropriate suffix for merging.
        # Use the str attribute to guarantee that the key was properly read as a string instead
        # of other type. Then, there will be no concatenation errors:
        suffix_right = str(suffix_right)
        # Concatenate an underscore on the left of suffix_left to obtain the suffix for merging.
        suffix_right = "_" + suffix_right
        
        # Create the tuple of suffixes:
        SUFFIXES = (suffix_left, suffix_right)
        
        # Apply the merge method to update the left_df my merging it to right_df.
        # The merged left_df will continue to be update on the following iterations:
        # notice that we could specify the arguments left_on = 'predictive_features',
        # and right_on = 'predictive_features'. Since the columns have the same name,
        # we omit this parameter and simply specify on = 'predictive_features'
        # We also modify the standard 'inner' to 'outer' join, so that no entries are
        # removed at first (pandas "outer" is a full outer join: it keeps all rows from
        # both dataframes)
        df_left = df_left.merge(df_right, on = 'predictive_features', how = "outer", suffixes = SUFFIXES)
        
        # Update the left_suffix as _merge_i. Since i is an integer, we again use the
        # str attribute to convert it to a string. So the string concatenation is allowed:
        suffix_left = "_merge_" + str(i) 
    
    # Now we must order the dataframe df_left in descending order by all of its columns except 
    # 'predictive_features'. This one will be in ascending (alphabetic) order.
    # The order of importance will be the same order of the dictionary, i.e., the order of the
    # dataframes passed. The last important column for sorting will be 'predictive_features'.
    # Then, we must create a list of columns in their order of importance, and a corresponding
    # list of booleans for the sorting order. This list will have value False for the columns
    # sorted in descending order; and value True for the column sorted in ascending order.
    
    # Start the list of columns importance and the list of booleans for ascending or descending
    # order:
    cols_importance = []
    asc_booleans = []
    
    # loop through all new columns of the dataframe:
    
    for column in df_left.columns:
        # loop through each element, named 'column' from the list df_left.columns (list of columns
        # of the merged dataframe):
        
        # check if the column is different from 'predictive_features'. 
        # If it is, add it to col_importance list and add a False value to asc_booleans list:
        if (column != 'predictive_features'):
            
            cols_importance.append(column)
            asc_booleans.append(False)
    
    # Now, we finished adding the columns to sort in descending order.
    # Then, we simply append 'predictive_features' to the cols_importance list, and append True
    # to the asc_booleans list:
    cols_importance.append('predictive_features')
    asc_booleans.append(True)
    
    # Now we can sort the df_left dataframe using the pandas sort_values method:
    # Since we are sorting by a subset, we pass the list of columns to by instead of passing a simple
    # string (case when we are sorting by a single column); and instead of passing a simple boolean
    # to ascending (case for single column)
    df_left = df_left.sort_values(by = cols_importance, ascending = asc_booleans)
    #sort by the cols_importance list; ascending is set on asc_booleans: if its True, column is 
    # sorted in ascending order; if it is False, sorting is performed in descending order.
    
    # Check if missing values should be dropped:
    if (eliminate_non_correspondence): # runs if the boolean is True:
        # drop rows (axis = 0) with missing values:
        # axis = 1 would drop columns containing missing values
        df_cleaned = df_left.dropna(axis = 0)
    
    # Now, reset index positions:
    df_left = df_left.reset_index(drop = True)
    
    # If limit_of_ranked_features is not None, use the head method to limit the output
    # from the dataframe:
    
    if not (limit_of_ranked_features is None):
        
        df_left = df_left.head(limit_of_ranked_features)
        # The dataframe will have only the 'limit_of_ranked_features' first entries
        print(f"General feature ranking successfully returned. It was limited to {limit_of_ranked_features}. Check the new dataframe:\n")
        print(df_left)
        
        return df_left
    
    else:
        
        print("General feature ranking successfully returned. Check the 20 first features:\n")
        print(df_left.head(20))
        
        return df_left

# **Function for calculating metrics for regression models**

In [ ]:
def regression_models_metrics (model_object, X_train, y_train, X_test = None, y_test = None):
    
    import numpy as np
    import pandas as pd
    from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
    from sklearn.neural_network import MLPRegressor
    from sklearn.neural_network import MLPClassifier
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.ensemble import RandomForestClassifier
    from keras.layers import Dense
    from keras.models import Sequential
    from xgboost import XGBRegressor
    from xgboost import XGBClassifier
    from sklearn.metrics import mean_squared_error
    from sklearn.metrics import r2_score
    from sklearn.metrics import mean_absolute_error
    
    # model_object: object containing the model that will be analyzed. e.g.
    # model_object = elastic_net_linear_reg_model
    # X_train = subset of predictive variables (dataframe).
    # y_train = subset of response variable (series).
    # X_test = subset of predictive variables (test dataframe, in case the 
    # original one was split into train and test).
    # y_test = subset of response variable (test series, in case the 
    # original one was split into train and test).
    
    y_pred = model_object.predict(X_train)
    
    # Start a dictionary for storing the metrics:
    metrics_dict = {}
    
    #METRICS FOR TRAINING
    
    #mean squared error- MSE
    mse_train = mean_squared_error(y_train, y_pred)
    # Save this metric in the dictionary:
    metrics_dict.update({'MSE_train': mse_train})
    print(f"Mean squared error (MSE) for training = {mse_train}")
    
    #Root mean squared error - RMSE
    rmse_train = np.sqrt(mse_train)
    metrics_dict.update({'RMSE_train': rmse_train})
    print(f"Root mean squared error (RMSE) for training = {rmse_train}")
    
    #R²
    val_r2 = r2_score(y_train, y_pred)
    metrics_dict.update({'R2_train': val_r2})
    print(f"Correlation coefficient R2 for training = {val_r2}")

    # n_size_train = number of sample size
    # k_model = number of independent variables of the defined model
    
    # the number of predictive features is the length of the list of the predictors:
    # Use the list attribute to guarantee that it is a list:
    k_model = len(list(X_train.columns)) # numer of X columns (variables)

    n_size_train = len(y_train)
    #numer of rows

    Adj_r2_train = 1 - (1 - val_r2)*(n_size_train - 1)/(n_size_train - k_model - 1)
    metrics_dict.update({'Adj_R2_train': Adj_r2_train})
    print(f"Adjusted coefficient of correlation Adj R2 for training = {Adj_r2_train}")
    
    if not ((X_test is None) & (y_test is None)):
        
        # calculate the metrics for the tests:
        y_pred_test = model_object.predict(X_test)
        
        #METRICS FOR TESTING
        #mean squared error- MSE
        mse_test = mean_squared_error(y_test, y_pred_test)
        # Save this metric in the dictionary:
        metrics_dict.update({'MSE_test': mse_test})
        print(f"Mean squared error (MSE) for testing = {mse_test}")
        #Root mean squared error - RMSE
        rmse_test = np.sqrt(mse_test)
        metrics_dict.update({'RMSE_test': rmse_test})
        print(f"Root mean squared error (RMSE) for testing = {rmse_test}")
        #R²
        val_r2_test = r2_score(y_test, y_pred_test)
        metrics_dict.update({'R2_test': val_r2_test})
        print(f"Correlation coefficient R2 for testing = {val_r2_test}")
        
        n_size_test = len(y_test)
        
        Adj_r2_test = 1 - (1 - val_r2_test)*(n_size_test - 1)/(n_size_test - k_model - 1)
        metrics_dict.update({'Adj_R2_test': Adj_r2_test})
        print(f"Adjusted coefficient of correlation Adj R2 for testing = {Adj_r2_test}")
    
    print("Dictionary with metrics returned as metrics_dict")
    
    return metrics_dict

# **Function for making predictions with the models**

In [ ]:
def make_model_predictions (model_object, X, predict_for = 'subset', concatenate_predictions_with_X = True, col_with_predictions_suffix = None):
    
    import numpy as np
    import pandas as pd
    from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
    from sklearn.neural_network import MLPRegressor
    from sklearn.neural_network import MLPClassifier
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.ensemble import RandomForestClassifier
    from keras.layers import Dense
    from keras.models import Sequential
    from xgboost import XGBRegressor
    from xgboost import XGBClassifier
    from sklearn.metrics import mean_squared_error
    from sklearn.metrics import r2_score
    from sklearn.metrics import mean_absolute_error
    
    # predict_for = 'subset' or predict_for = 'single_entry'
    
    # X = subset of predictive variables (dataframe).
    # If PREDICT_FOR = 'single_entry', X should be a list of parameters values.
    # e.g. X = [1.2, 3, 4] (dot is the decimal case separator, comma separate values). 
    # Notice that the list should contain only the numeric values, in the same order of the
    # correspondent columns.
    # If PREDICT_FOR = 'subset' (prediction for multiple entries), X should be a dataframe 
    # (subset) of the parameters values, as usual.
    
    # model_object: object containing the model that will be analyzed. e.g.
    # model_object = elastic_net_linear_reg_model
    
    # concatenate_predictions_with_X = True to concatenate the predicted values with X. 
    # For a single entry, it will be appended as an element from the list. For multiple entries, 
    # it will be added as a column.
    # If concatenate_predictions_with_X = False, the prediction will be returned as a single variable value
    # (single entry), or as a series (prediction for a subset).
    
    # col_with_predictions_suffix = None. If you want to add a new column, you can declare this
    # parameter as string with a suffix for identifying the model. If no suffix is added, the new
    # column will be named 'y_pred'.
    # e.g. col_with_predictions_suffix = '_keras' will create a column named "y_pred_keras". This
    # parameter is useful when working with multiple models. Always start the suffix with underscore
    # "_" so that no blank spaces are added; the suffix will not be merged to the column; and there
    # will be no confusion with the dot (.) notation for methods, JSON attributes, etc.
    
    if (predict_for == 'single_entry'):
        
        print("Making prediction for a single entry. X must be a list with values in the order of the correspondent columns of the dataset.")
        
        # Get reshaped list for making the prediction:
        X_reshaped = np.reshape(np.array(X), (-1, 1))
        
        y_pred = model_object.predict(X_reshaped)
        print(f"Output value predicted for the entry parameters = {y_pred}\n")
        print("Attention: for classification, this output may be either a class or a probability. Check the model documentation for verifying it.")
        
        if (concatenate_predictions_with_X == True):
            
            # concatenate the predicted values with X. For a single entry, it will be appended as a new
            # element from the list. For multiple entries, it will be added as a column
            appended_list = X.append(y_pred)
            
            print("The prediction was added as a new element of the list X, and the appended list was returned.")
            
            return appended_list
        
        else:
            
            print("Returning only the predicted value.")
            
            return y_pred
    
    else:
        
        # prediction for a subset
        y_pred = model_object.predict(X)
        
        if (concatenate_predictions_with_X == True):
            
            # concatenate the predicted values with X. For a single entry, it will be appended as a new
            # element from the list. For multiple entries, it will be added as a column
            
            # check if there is a suffix:
            if not (col_with_predictions_suffix is None):
                
                # if there is a suffix, concatenate it to 'y_pred':
                col_name = "y_pred" + col_with_predictions_suffix
            
            else:
                # the name of the new column is simply 'y_pred'
                col_name = "y_pred"
            
            # Set a local copy of the dataframe to manipulate:
            X_copy = X
            
            # Add the predictions as the new column named col_name:
            X_copy[col_name] = y_pred
            
            print(f"The prediction was added as the new column {col_name} of the dataframe X, and this dataframe was returned.")
            
            return X_copy
        
        else:
            
            print("Returning only the predicted values.")
            
            return y_pred

# **Function for importing or exporting models and dictionaries**

In [3]:
def import_export_model_or_dict (action = 'import', objects_manipulated = 'model_only', model_file_name = None, dictionary_file_name = None, directory_path = '/', model_type = 'keras', dict_to_export = None, model_to_export = None, use_colab_memory = False):
    
    import os
    import pickel as pkl
    import dill
    from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
    from sklearn.neural_network import MLPRegressor
    from sklearn.neural_network import MLPClassifier
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.ensemble import RandomForestClassifier
    from keras.layers import Dense
    from keras.models import Sequential
    from xgboost import XGBRegressor
    from xgboost import XGBClassifier
    from statsmodels.tsa.arima.model import ARIMA
    from statsmodels.tsa.arima.model import ARIMAResults
    from keras.models import load_model
    from google.colab import files
    
    # action = 'import' for importing a model and/or a dictionary;
    # action = 'export' for exporting a model and/or a dictionary.
    
    # objects_manipulated = 'model_only' if only a model will be manipulated.
    # objects_manipulated = 'dict_only' if only a dictionary will be manipulated.
    # objects_manipulated = 'model_and_dict' if both a model and a dictionary will be
    # manipulated.
    
    #model_file_name: string with the name of the file containing the model (for 'import');
    # or of the name that the exported file will have (for 'export')
    # e.g. model_file_name = 'model'
    # WARNING: Do not add the file extension.
    # Keep it in quotes. Keep model_file_name = None if no model will be manipulated.
    
    # dictionary_file_name: string with the name of the file containing the dictionary 
    # (for 'import');
    # or of the name that the exported file will have (for 'export')
    # e.g. dictionary_file_name = 'history_dict'
    # WARNING: Do not add the file extension.
    # Keep it in quotes. Keep DICTIONARY_FILE_NAME = None if no 
    # dictionary will be manipulated.
    
    # DIRECTORY_PATH: path of the directory where the model will be saved,
    # or from which the model will be retrieved. If no value is provided,
    # the DIRECTORY_PATH will be the root: "/"
    # Notice that the model and the dictionary must be stored in the same path.
    # If a model and a dictionary will be exported, they will be stored in the same
    # DIRECTORY_PATH.
    
    # model_type: This parameter has effect only when a model will be manipulated.
    # model_type: 'keras' for deep learning keras/ tensorflow models with extension .h5
    # model_type = 'sklearn' for models from scikit-learn (non-deep learning)
    # model_type = 'xgb' for XGBoost models (non-deep learning)
    # model_type = 'arima' for ARIMA model (Statsmodels)
    
    # dict_to_export and model_to_export: 
    # These two parameters have effect only when ACTION == 'export'. In this case, they
    # must be declared. If ACTION == 'export', keep:
    # dict_to_export = None, 
    # model_to_export = None
    # If one of these objects will be exported, substitute None by the name of the object
    # e.g. if your model is stored in the global memory as 'keras_model' declare:
    # model_to_export = keras_model. Notice that it must be declared without quotes, since
    # it is not a string, but an object.
    # For exporting a dictionary named as 'dict':
    # dict_to_export = dict
    
    # use_colab_memory: this parameter has only effect when using Google Colab (or it will
    # raise an error). Set as use_colab_memory = True if you want to use the instant memory
    # from Google Colaboratory: you will update or download the file and it will be available
    # only during the time when the kernel is running. It will be excluded when the kernel
    # dies, for instance, when you close the notebook.
    
    # If action == 'export' and use_colab_memory == True, then the file will be downloaded
    # to your computer (running the cell will start the download).
    
    # Check the directory path
    if (directory_path is None):
        # set as the root:
        directory_path = "/"
        
        
    bool_check1 = (objects_manipulated != 'model_only')
    # bool_check1 == True if a dictionary will be manipulated
    
    bool_check2 = (objects_manipulated != 'dict_only')
    # bool_check1 == True if a dictionary will be manipulated
    
    if (bool_check1 == True):
        #manipulate a dictionary
        
        if (dictionary_file_name is None):
            print("Please, enter a name for the dictionary.")
            return "error1"
        
        else:
            # Create the file path for the dictionary:
            dict_path = os.path.join(directory_path, dictionary_file_name)
            # Extract the file extension
            dict_extension = 'pkl'
            #concatenate:
            dict_path = dict_path + "." + dict_extension
            
    
    if (bool_check2 == True):
        #manipulate a model
        
        if (model_file_name is None):
            print("Please, enter a name for the model.")
            return "error1"
        
        else:
            # Create the file path for the dictionary:
            model_path = os.path.join(directory_path, model_file_name)
            # Extract the file extension
            
            #check model_type:
            if (model_type == 'keras'):
                model_extension = 'h5'
            
            elif (model_type == 'sklearn'):
                model_extension = 'dill'
                #it could be 'pkl', though
            
            elif (model_type == 'xgb'):
                model_extension = 'json'
                #it could be 'ubj', though
            
            elif (model_tyoe == 'arima'):
                model_extension = 'pkl'
            
            else:
                print("Enter a valid model_type: keras, sklearn_xgb, or arima.")
                return "error2"
            
            #concatenate:
            model_path = model_path +  "." + model_extension
            
    # Now we have the full paths for the dictionary and for the model.
    
    if (action == 'import'):
        
        if (use_colab_memory == True):
            
            print("Click on the button for file selection and select the files from your machine that will be uploaded in the Colab environment.")
            print("Warning: the files will be removed from Colab memory after the Kernel dies or after the notebook is closed.")
            # this functionality requires the previous declaration:
            ## from google.colab import files
            colab_files_dict = files.upload()
            # The files are stored into a dictionary called colab_files_dict where the keys
            # are the names of the files and the values are the files themselves.
            ## e.g. if you upload a single file named "dictionary.pkl", the dictionary will be
            ## colab_files_dict = {'dictionary.pkl': file}, where file is actually a big string
            ## representing the contents of the file. The length of this value is the size of the
            ## uploaded file, in bytes.
            ## To access the file is like accessing a value from a dictionary: 
            ## d = {'key1': 'val1'}, d['key1'] == 'val1'
            ## we simply declare the key inside brackets and quotes, the same way we would do for
            ## accessing the column of a dataframe.
            ## In this example, colab_files_dict['dictionary.pkl'] access the content of the 
            ## .pkl file, and len(colab_files_dict['dictionary.pkl']) is the size of the .pkl
            ## file in bytes.
            ## To check the dictionary keys, apply the method .keys() to the dictionary (with empty
            ## parentheses): colab_files_dict.keys()
            
            for key in colab_files_dict.keys():
                #loop through each element of the list of keys of the dictionary
                # (list colab_files_dict.keys()). Each element is named 'key'
                print(f"User uploaded file {key} with length {len(colab_files_dict[key])} bytes.")
                # The key is the name of the file, and the length of the value
                ## correspondent to the key is the file's size in bytes.
                ## Notice that the content of the uploaded object must be passed 
                ## as argument for a proper function to be interpreted. 
                ## For instance, the content of a xlsx file should be passed as
                ## argument for Pandas .read_excel function; the pkl file must be passed as
                ## argument for pickle.
                ## e.g., if you uploaded 'table.xlsx' and stored it into colab_files_dict you should
                ## declare df = pd.read_excel(colab_files_dict['table.xlsx']) to obtain a dataframe
                ## df from the uploaded table. Notice that is the value, not the key, that is the
                ## argument.
        
        if (bool_check1 == True):
            #manipulate a dictionary
            if (use_colab_memory == True):
                key = dictionary_file_name + "." + dict_extension
                #Use the key to access the file content, and pass the file content
                # to pickle:
                with open(colab_files_dict[key], 'rb') as opened_file:
            
                    imported_dict = pkl.load(opened_file)
                    # The structure imported_dict = pkl.load(open(colab_files_dict[key], 'rb')) relies 
                    # on the GC to close the file. That's not a good idea: If someone doesn't use 
                    # CPython the garbage collector might not be using refcounting (which collects 
                    # unreferenced objects immediately) but e.g. collect garbage only after some time.
                    # Since file handles are closed when the associated object is garbage collected or 
                    # closed explicitly (.close() or .__exit__() from a context manager) the file 
                    # will remain open until the GC kicks in.
                    # Using 'with' ensures the file is closed as soon as the block is left - even if 
                    # an exception happens inside that block, so it should always be preferred for any 
                    # real application.
                    # source: https://stackoverflow.com/questions/39447362/equivalent-ways-to-json-load-a-file-in-python

                print(f"Dictionary {key} successfully imported to Colab environment.")
            
            else:
                #standard method
                with open(dict_path, 'rb') as opened_file:
            
                    imported_dict = pkl.load(opened_file)
                
                # 'rb' stands for read binary (read mode). For writing mode, 'wb', 'write binary'
                print(f"Dictionary successfully imported from {dict_path}.")
                
        if (bool_chek2 == True):
            #manipulate a model
            # select the proper model
        
            if (model_type == 'keras'):
                
                if (use_colab_memory == True):
                    key = model_file_name + "." + model_extension
                    model = load_model(colab_files_dict[key])
                    print(f"Keras/TensorFlow model: {key} successfully imported to Colab environment.")
            
                else:
                    #standard method
                    # We previously declared:
                    # from keras.models import load_model
                    model = load_model(model_path)
                    print(f"Keras/TensorFlow model successfully imported from {model_path}.")

            elif (model_type == 'sklearn'):
                
                if (use_colab_memory == True):
                    key = model_file_name + "." + model_extension
                    
                    with open(colab_files_dict[key], 'rb') as opened_file:
            
                        model = dill.load(opened_file)
                    
                    print(f"Scikit-learn model: {key} successfully imported to Colab environment.")
            
                else:
                    #standard method
                    with open(model_path, 'rb') as opened_file:
            
                        model = dill.load(opened_file)
                
                    print(f"Scikit-learn model successfully imported from {model_path}.")
                    # For loading a pickle model:
                    ## model = pkl.load(open(model_path, 'rb'))
                    # 'rb' stands for read binary (read mode). For writing mode, 'wb', 'write binary'

            elif (model_type == 'xgb'):
                
                if (use_colab_memory == True):
                    key = model_file_name + "." + model_extension
                    model = load_model(colab_files_dict[key])
                    print(f"XGBoost model: {key} successfully imported to Colab environment.")
            
                else:
                    #standard method
                    model = load_model(model_path)
                    print(f"XGBoost model successfully imported from {model_path}.")
                    # model.load_model("model.json") or model.load_model("model.ubj")
                    # .load_model is a method from xgboost object

            elif (model_type == 'arima'):
                
                if (use_colab_memory == True):
                    key = model_file_name + "." + model_extension
                    model = ARIMAResults.load(colab_files_dict[key])
                    print(f"ARIMA model: {key} successfully imported to Colab environment.")
            
                else:
                    #standard method
                    # We previously declared:
                    # from statsmodels.tsa.arima.model import ARIMAResults
                    model = ARIMAResults.load(model_path)
                    print(f"ARIMA model successfully imported from {model_path}.")
            
            if (objects_manipulated == 'model_only'):
                # only the model should be returned
                return model
            
            elif (objects_manipulated == 'dict_only'):
                # only the dictionary should be returned:
                return imported_dict
            
            else:
                # Both objects are returned:
                return model, imported_dict

    
    elif (action == 'export'):
        
        #Let's export the models or dictionary:
        if (use_colab_memory == True):
            print("The files will be downloaded to your computer.")
        
        if (bool_check1 == True):
            #manipulate a dictionary
            if (use_colab_memory == True):
                ## Download the dictionary
                key = dictionary_file_name + "." + dict_extension
                
                with open(key, 'wb') as opened_file:
            
                    pkl.dump(dict_to_export, opened_file)
                
                # this functionality requires the previous declaration:
                ## from google.colab import files
                files.download(key)
                
                print(f"Dictionary {key} successfully downloaded from Colab environment.")
            
            else:
                #standard method 
                with open(dict_path, 'wb') as opened_file:
            
                    pkl.dump(dict_to_export, opened_file)
                
                #to save the file, the mode must be set as 'wb' (write binary)
                print(f"Dictionary successfully exported as {dict_path}.")
                
        if (bool_chek2 == True):
            #manipulate a model
            # select the proper model
        
            if (model_type == 'keras'):
                
                if (use_colab_memory == True):
                    ## Download the model
                    key = model_file_name + "." + model_extension
                    model_to_export.save(key)
                    files.download(key)
                    print(f"Keras/TensorFlow model: {key} successfully downloaded from Colab environment.")
            
                else:
                    #standard method
                    model_to_export.save(model_path)
                    print(f"Keras/TensorFlow model successfully exported as {model_path}.")

            elif (model_type == 'sklearn'):
                
                if (use_colab_memory == True):
                    ## Download the model
                    key = model_file_name + "." + model_extension
                    
                    with open(key, 'wb') as opened_file:

                        dill.dump(model_to_export, opened_file)
                    
                    #to save the file, the mode must be set as 'wb' (write binary)
                    files.download(key)
                    print(f"Scikit-learn model: {key} successfully downloaded from Colab environment.")
            
                else:
                    #standard method
                    with open(model_path, 'wb') as opened_file:

                        dill.dump(model_to_export, opened_file)
                    
                    print(f"Scikit-learn model successfully exported as {model_path}.")
                    # For exporting a pickle model:
                    ## pkl.dump(model_to_export, open(model_path, 'wb'))
            
            elif (model_type == 'xgb'):
                
                if (use_colab_memory == True):
                    ## Download the model
                    key = model_file_name + "." + model_extension
                    model_to_export.save_model(key)
                    files.download(key)
                    print(f"XGBoost model: {key} successfully downloaded from Colab environment.")
            
                else:
                    #standard method
                    model_to_export.save_model(model_path)
                    print(f"XGBoost model successfully exported as {model_path}.")
                    # For exporting a pickle model:
                    ## pkl.dump(model_to_export, open(model_path, 'wb'))
            
            elif (model_type == 'arima'):
                
                if (use_colab_memory == True):
                    ## Download the model
                    key = model_file_name + "." + model_extension
                    model_to_export.save(key)
                    files.download(key)
                    print(f"ARIMA model: {key} successfully downloaded from Colab environment.")
            
                else:
                    #standard method
                    model_to_export.save(model_path)
                    print(f"ARIMA model successfully exported as {model_path}.")
        
        print("Export of files completed.")
    
    else:
        print("Enter a valid action, import or export.")

# **Function for exporting the dataframe**

In [ ]:
def export_dataframe (dataframe_to_be_exported, new_file_name_with_csv_extension, file_directory_path = None, export_to_s3_bucket = False, s3_bucket_name = None, desired_s3_file_name_with_csv_extension = None):
    
    import os
    import boto3
    #boto3 is AWS S3 Python SDK
    import pandas as pd
    
    ## WARNING: all file extensions should be .csv for this function
    
    # FILE_DIRECTORY_PATH - (string, in quotes): input the path of the directory 
    # (e.g. folder path) where the file is stored. e.g. FILE_DIRECTORY_PATH = "/" 
    # or FILE_DIRECTORY_PATH = "/folder"
    # If you want to export the file to AWS S3, this parameter will have no effect.
    # In this case, you can set FILE_DIRECTORY_PATH = None

    # NEW_FILE_NAME_WITH_CSV_EXTENSION - (string, in quotes): input the name of the 
    # file with the  extension. e.g. FILE_NAME_WITH_CSV_EXTENSION = "file.csv"
    
    # export_to_s3_bucket = False. Alternatively, set as True to export the file to an
    # AWS S3 Bucket.

    ## The following parameters have effect only when export_to_s3_bucket == True:

    ## This parameter is obbligatory to access an AWS S3 bucket. Substitute it for a string
    # with the bucket's name. e.g. S3_BUCKET_NAME = "aws-bucket-1" access a bucket named as
    # "aws-bucket-1"

    # The name desired for the object stored in S3 (string, in quotes). 
    # Keep it None to set it equals to new_file_name_with_csv_extension. 
    # Alternatively, set it as a string analogous to new_file_name_with_csv_extension. 
    # e.g. desired_s3_file_name_with_csv_extension = "S3_file.csv"
    
    if (export_to_s3_bucket == True):
        
        if (desired_s3_file_name_with_csv_extension is None):
            #Repeat new_file_name_with_extension
            desired_s3_file_name_with_csv_extension = new_file_name_with_csv_extension
        
        # If the bucket name was provided, start the session. If not, print an error
        # message:
        
        if (s3_bucket_name is None):
            
            print("Please, provide a valid S3 Bucket name to download from.")
        
        else:
        
            # start S3 client:
            print("Starting AWS S3 client.")
        
            # Let's export the file to a AWS S3 (simple storage service) bucket
            # instantiate S3 client and upload to s3
            s3_client = boto3.resource('s3')
            
            # Create a local copy of the file on the root.
            local_copy_path = os.path.join("/", new_file_name_with_csv_extension)
            dataframe_to_be_exported.to_csv(local_copy_path, index = False)
            
            print("Local copy of the dataframe created on the root path to export to S3.")
            print("Simply delete this file from the root path if you only want to keep the S3 version.")
            
            # Upload this local copy to S3:
            try:
                response = s3_client.meta.client.upload_file(local_copy_path, s3_bucket_name, desired_s3_file_name_with_extension)
            
            except ClientError as e:
                logging.error(e)
                return False
            
            print(f"{desired_s3_file_name_with_csv_extension} successfully exported to {s3_bucket_name} AWS S3 bucket.")
            return True
            # Check AWS Documentation:
            # https://boto3.amazonaws.com/v1/documentation/api/latest/guide/s3-uploading-files.html
            
            # Notice: if you wanted to authenticate directly from Python code, you could use
            # the following code, instead:        
            # ACCESS_KEY = 'access_key_ID'
            # PASSWORD_KEY = 'password_key'
            # s3_client = boto3.client('s3', aws_access_key_id = ACCESS_KEY, aws_secret_access_key = PASSWORD_KEY)
            # s3_client.upload_file(local_copy_path, s3_bucket_name, desired_s3_file_name_with_extension)
            
    else :
        # Do not export to AWS S3. Export to other path.
        # Create the complete file path:
        file_path = os.path.join(file_directory_path, new_file_name_with_csv_extension)

        dataframe_to_be_exported.to_csv(file_path, index = False)

        print(f"Dataframe {new_file_name_with_csv_extension} exported as \'{file_path}\'.")
        print("Warning: if there was a file in this file path, it was replaced by the exported dataframe.")

# **Function for downloading a file from Google Colab or AWS S3 to the local machine or uploading a file from the machine to S3 or to Colab's instant memory**

In [2]:
def download_or_upload_file (source = 'aws', action = 'download', object_to_download_from_colab = None, s3_bucket_name = None, local_path_of_storage = '/', file_name_with_extension = None):
    
    import os
    import boto3
    # boto3 is AWS S3 Python SDK
    from google.colab import files
    
    # source = 'google' for downloading from (or uploading to) Google Colab's instant memory;
    # source = 'aws' for downloading from (or uploading to) an AWS S3 bucket.
    
    # action = 'download' to download the file to the local machine
    # action = 'upload' to upload a file from local machine to AWS S3 or to
    # Google Colab's instant memory
    
    # object_to_download_from_colab = None. This option has effect only when
    # source == 'google'. In this case, this parameter is obbligatory. 
    # Declare as object_to_download_from_colab the object that you want to download.
    # Since it is an object and not a string, it should not be declared in quotes.
    # e.g. to download a dictionary named dict, object_to_download_from_colab = dict.
    # To download a dataframe named df, declare object_to_download_from_colab = df.
    # To export a model named keras_model, declare object_to_download_from_colab = keras_model
    
    ## THE FOLLOWING PARAMETERS HAVE EFFECT ONLY WHEN source == 'aws'
    
    # s3_bucket_name = None.
    ## This parameter is obbligatory to access an AWS S3 bucket. Substitute it for a string
    # with the bucket's name. e.g. s3_bucket_name = "aws-bucket-1" access a bucket named as
    # "aws-bucket-1"
    
    # LOCAL_PATH_OF_STORAGE: path of the local computer environment 
    # to which the S3 bucket contents will be downloaded (ACTION == 'download'); or
    # path of the folder containing the file that will be uploaded in S3 (ACTION = 'upload'). 
    # If it is None, or if LOCAL_PATH_OF_STORAGE = '/', files 
    # will be imported to the root path. Alternatively, input the path as a string 
    # (in quotes).
    # Examples: LOCAL_PATH_OF_STORAGE = '/copied_s3_bucket'; 
    # LOCAL_PATH_OF_STORAGE = "/My_folder"; LOCAL_PATH_OF_STORAGE = "/Users/Me/Documents/"
    # Notice that only the directories should be declared: do not include the file name and
    # its extension.
    
    # file_name_with_extension: string, in quotes, containing the file name which will be
    # downloaded from S3; or uploaded from S3, followed by its extension. 
    ## This parameter is obbligatory when source == 'aws'
    # Examples:
    # file_name_with_extension = 'Screen_Shot.png'; file_name_with_extension = 'dataset.csv',
    # file_name_with_extension = "dictionary.pkl", file_name_with_extension = "model.h5",
    # file_name_with_extension = 'doc.pdf', file_name_with_extension = 'model.dill'

    if (source == 'google'):
        
        if (action == 'upload'):
            
            print("Click on the button for file selection and select the files from your machine that will be uploaded in the Colab environment.")
            print("Warning: the files will be removed from Colab memory after the Kernel dies or after the notebook is closed.")
            # this functionality requires the previous declaration:
            ## from google.colab import files
            
            colab_files_dict = files.upload()
            
            # The files are stored into a dictionary called colab_files_dict where the keys
            # are the names of the files and the values are the files themselves.
            ## e.g. if you upload a single file named "dictionary.pkl", the dictionary will be
            ## colab_files_dict = {'dictionary.pkl': file}, where file is actually a big string
            ## representing the contents of the file. The length of this value is the size of the
            ## uploaded file, in bytes.
            ## To access the file is like accessing a value from a dictionary: 
            ## d = {'key1': 'val1'}, d['key1'] == 'val1'
            ## we simply declare the key inside brackets and quotes, the same way we would do for
            ## accessing the column of a dataframe.
            ## In this example, colab_files_dict['dictionary.pkl'] access the content of the 
            ## .pkl file, and len(colab_files_dict['dictionary.pkl']) is the size of the .pkl
            ## file in bytes.
            ## To check the dictionary keys, apply the method .keys() to the dictionary (with empty
            ## parentheses): colab_files_dict.keys()
            
            for key in colab_files_dict.keys():
                #loop through each element of the list of keys of the dictionary
                # (list colab_files_dict.keys()). Each element is named 'key'
                print(f"User uploaded file {key} with length {len(colab_files_dict[key])} bytes.")
                # The key is the name of the file, and the length of the value
                ## correspondent to the key is the file's size in bytes.
                ## Notice that the content of the uploaded object must be passed 
                ## as argument for a proper function to be interpreted. 
                ## For instance, the content of a xlsx file should be passed as
                ## argument for Pandas .read_excel function; the pkl file must be passed as
                ## argument for pickle.
                ## e.g., if you uploaded 'table.xlsx' and stored it into colab_files_dict you should
                ## declare df = pd.read_excel(colab_files_dict['table.xlsx']) to obtain a dataframe
                ## df from the uploaded table. Notice that is the value, not the key, that is the
                ## argument.
                
                print("The uploaded files are stored into a dictionary object named as colab_files_dict.")
                print("Each key from this dictionary is the name of an uploaded file. The value correspondent to that key is the file itself.")
                print("The structure of a general Python dictionary is dict = {\'key1\': value1}. To access value1, declare file = dict[\'key1\'], as if you were accessing a column from a dataframe.")
                print("Then, if you uploaded a file named \'table.xlsx\', you can access this file as:")
                print("uploaded_file = colab_files_dict[\'table.xlsx\']")
                print("Notice, though, that the object uploaded_file is the whole file content, not a Python object already converted. To convert to a Python object, pass this element as argument for a proper function or method.")
                print("In this example, to convert the object uploaded_file to a dataframe, Pandas pd.read_excel function could be used. In the following line, a df dataframe object is obtained from the uploaded file:")
                print("df = pd.read_excel(uploaded_file)")
        
        elif (action == 'download'):
            
            if (object_to_download_from_colab is None):
                
                #No object was declared
                print("Please, inform an object to download. Since it is an object, not a string, it should not be declared in quotes.")
            
            else:
                
                print("The file will be downloaded to your computer.")

                files.download(object_to_download_from_colab)

                print(f"File {object_to_download_from_colab} successfully downloaded from Colab environment.")

        else:
            
            print("Please, select a valid action, download or upload.")
          
    elif (source == 'aws'):
        
        # Notice: if you wanted to authenticate directly from Python code, you could use
        # the following code, instead for starting the client:
        
        # ACCESS_KEY = 'access_key_ID'
        # PASSWORD_KEY = 'password_key'
        # s3_client = boto3.client('s3', aws_access_key_id = ACCESS_KEY, aws_secret_access_key = PASSWORD_KEY)
        # Nextly, the code is the same.
        
        
        # If the path to store is None, also import the bucket content to root path;
        # or upload the file from root path to the bucket
        if (local_path_of_storage is None):
            
            local_path_of_storage = '/'
        
        # If the bucket name was provided, start the session. If not, print an error
        # message. The same for the file name with extension:
        
        if (s3_bucket_name is None):
            
            print("Please, provide a valid S3 Bucket name.")
        
        elif (file_name_with_extension is None):
            
            print("Please, provide a valid file name with its extension. e.g. \'dataset.csv\'.")
        
        else:
            
            # Obtain the full file path from which the file will be uploaded to S3; or to
            # which the file will be downloaded from S3:
            file_path = os.path.join(local_path_of_storage, file_name_with_extension)
            
            # Start S3 client:
            s3_client = boto3.resource('s3')
            
            print("Starting AWS S3 client.")
            
            if (action == 'upload'):
                
                s3_client.Object(s3_bucket_name, file_name_with_extension).\
                    upload_file(Filename = file_path)
                
                print(f"File {file_name_with_extension} successfully uploaded to AWS S3 {s3_bucket_name} bucket.")
            
            elif (action == 'download'):

                print("The file will be downloaded to your computer.")
                
                s3_client.Object(s3_bucket_name, file_name_with_extension).download_file(file_path)
                
                print(f"File {file_name_with_extension} successfully downloaded from AWS S3 {s3_bucket_name} bucket.")

            else:

                print("Please, select a valid action, download or upload.")

    else:
        
        print("Select a valid source: \'google\' for mounting Google Drive; or \'aws\' for accessing AWS S3 Bucket.")

## **Call the functions**

### **Mounting Google Drive or S3 (AWS Simple Storage Service) bucket**

In [ ]:
SOURCE = 'aws'
# SOURCE = 'google' for mounting the google drive;
# SOURCE = 'aws' for accessing an AWS S3 bucket

## THE FOLLOWING PARAMETERS HAVE EFFECT ONLY WHEN SOURCE == 'aws':

PATH_TO_STORE_IMPORTED_S3_BUCKET = '/'
# PATH_TO_STORE_IMPORTED_S3_BUCKET: path of the Python environment to which the
# S3 bucket contents will be imported. If it is None, or if 
# PATH_TO_STORE_IMPORTED_S3_BUCKET = '/', bucket will be imported to the root path. 
# Alternatively, input the path as a string (in quotes). e.g. 
# PATH_TO_STORE_IMPORTED_S3_BUCKET = '/copied_s3_bucket'

S3_BUCKET_NAME = 'name_of_aws_s3_bucket_to_be_accessed'
## This parameter is obbligatory to access an AWS S3 bucket. Substitute it for a string
# with the bucket's name. e.g. s3_bucket_name = "aws-bucket-1" access a bucket named as
# "aws-bucket-1"

S3_OBJECT_KEY_PREFFIX_FOLDER = None
# S3_OBJECT_KEY_PREFFIX_FOLDER = None. Keep it None or as an empty string 
# (S3_OBJECT_KEY_PREFFIX_FOLDER = '') to import the whole bucket content, instead of a 
# single object from it.
# Alternatively, set it as a string containing the subfolder from the bucket to import:
# Suppose that your bucket (admin-created) has four objects with the following object 
# keys: Development/Projects1.xls; Finance/statement1.pdf; Private/taxdocument.pdf; and
# s3-dg.pdf. 
# The s3-dg.pdf key does not have a prefix, so its object appears directly 
# at the root level of the bucket. If you open the Development/ folder, you see 
# the Projects.xlsx object in it.
# In summary, if the path of the file is: 'bucket/my_path/.../file.csv'
# where 'bucket' is the bucket's name, key_preffix = 'my_path/.../', without the
# 'file.csv' (file name with extension) last part.

mount_storage_system (source = SOURCE, path_to_store_imported_s3_bucket = PATH_TO_STORE_IMPORTED_S3_BUCKET, s3_bucket_name = S3_BUCKET_NAME, s3_obj_key_preffix = S3_OBJECT_KEY_PREFFIX_FOLDER)

### **Importing the dataset**

In [3]:
## WARNING: Use this function to load dataframes stored on Excel (xls, xlsx, etc), 
## JSON, txt, or CSV (comma separated values) files.

FILE_DIRECTORY_PATH = "/"
# FILE_DIRECTORY_PATH - (string, in quotes): input the path of the directory 
# (e.g. folder path) where the file is stored. e.g. FILE_DIRECTORY_PATH = "/" 
# or FILE_DIRECTORY_PATH = "/folder"

FILE_NAME_WITH_EXTENSION = "dataset.csv"
# FILE_NAME_WITH_EXTENSION - (string, in quotes): input the name of the file with the 
# extension. e.g. FILE_NAME_WITH_EXTENSION = "file.xlsx", or, 
# FILE_NAME_WITH_EXTENSION = "file.csv"

LOAD_TXT_FILE_WITH_JSON_FORMAT = False
# LOAD_TXT_FILE_WITH_JSON_FORMAT = False. Set LOAD_TXT_FILE_WITH_JSON_FORMAT = True 
# if you want to read a file with txt extension containing a text formatted as JSON 
# (but not saved as JSON).
# WARNING: if LOAD_TXT_FILE_WITH_JSON_FORMAT = True, all the JSON file parameters of the 
# function (below) must be set. If not, an error message will be raised.
    
HAS_HEADER = True
# HAS_HEADER = True if the the imported table has headers (row with columns names).
# Alternatively, HAS_HEADER = False if the dataframe does not have header.

## Parameters for loading txt or CSV files:

TXT_CSV_COL_SEP = "comma"
# TXT_CSV_COL_SEP = "comma" - This parameter has effect only when the file is a 'txt'
# or 'csv'. It informs how the different columns are separated.
# Alternatively, TXT_CSV_COL_SEP = "comma" for columns separated by comma (",")
# TXT_CSV_COL_SEP = "whitespace" for columns separated by simple spaces (" ").

SHEET_TO_LOAD = None
# SHEET_TO_LOAD - This parameter has effect only when for Excel files.
# keep SHEET_TO_LOAD = None not to specify a sheet of the file, so that the first sheet
# will be loaded.
# SHEET_TO_LOAD may be an integer or an string (inside quotes). SHEET_TO_LOAD = 0
# loads the first sheet (sheet with index 0); SHEET_TO_LOAD = 1 loads the second sheet
# of the file (index 1); SHEET_TO_LOAD = "Sheet1" loads a sheet named as "Sheet1".
# Declare a number to load the sheet with that index, starting from 0; or declare a
# name to load the sheet with that name.

## Parameters for loading JSON files:

JSON_RECORD_PATH = None
# JSON_RECORD_PATH (string): manipulate parameter 'record_path' from json_normalize method.
# Path in each object to list of records. If not passed, data will be assumed to 
# be an array of records. If a given field from the JSON stores a nested JSON (or a nested
# dictionary) declare it here to decompose the content of the nested data. e.g. if the field
# 'books' stores a nested JSON, declare, JSON_RECORD_PATH = 'books'

JSON_FIELD_SEPARATOR = "_"
# JSON_FIELD_SEPARATOR = "_" (string). Manipulates the parameter 'sep' from json_normalize method.
# Nested records will generate names separated by sep. 
# e.g., for JSON_FIELD_SEPARATOR = ".", {‘foo’: {‘bar’: 0}} -> foo.bar.
# Then, if a given field 'main_field' stores a nested JSON with fields 'field1', 'field2', ...
# the name of the columns of the dataframe will be formed by concatenating 'main_field', the
# separator, and the names of the nested fields: 'main_field_field1', 'main_field_field2',...

JSON_METADATA_PREFFIX_LIST = None
# JSON_METADATA_PREFFIX_LIST: list of strings (in quotes). Manipulates the parameter 
# 'meta' from json_normalize method. Fields to use as metadata for each record in resulting 
# table. Declare here the non-nested fields, i.e., the fields in the principal JSON. They
# will be repeated in the rows of the dataframe to give the metadata (context) of the rows.

# e.g. Suppose a JSON with the following structure: {'name': 'Mary', 'last': 'Shelley',
# 'books': [{'title': 'Frankestein', 'year': 1818}, {'title': 'Mathilda ', 'year': 1819},{'title': 'The Last Man', 'year': 1826}]},
# Here, there are nested JSONs in the field 'books'. The fields that are not nested
# are 'name' and 'last'.
# Then, JSON_RECORD_PATH = 'books'
# JSON_METADATA_PREFFIX_LIST = ['name', 'last']

#The dataframe will be stored in the object named 'dataset':
# Simply modify this object on the left of equality:
dataset = load_dataframe (file_directory_path = FILE_DIRECTORY_PATH, file_name_with_extension = FILE_NAME_WITH_EXTENSION, load_txt_file_with_json_format = LOAD_TXT_FILE_WITH_JSON_FORMAT, has_header = HAS_HEADER, txt_csv_col_sep = TXT_CSV_COL_SEP, sheet_to_load = SHEET_TO_LOAD, json_record_path = JSON_RECORD_PATH, json_field_separator = JSON_FIELD_SEPARATOR, json_metadata_preffix_list = JSON_METADATA_PREFFIX_LIST)

### **Converting JSON object to dataframe**

In [3]:
JSON_OBJ_TO_CONVERT = json_object #Alternatively: object containing the JSON to be converted

# JSON_OBJ_TO_CONVERT: object containing JSON, or string with JSON content to parse.
# Objects may be: string with JSON formatted text;
# list with nested dictionaries (JSON formatted);
# dictionaries, possibly with nested dictionaries (JSON formatted).

## Parameters for loading JSON files:

JSON_RECORD_PATH = None
# JSON_RECORD_PATH (string): manipulate parameter 'record_path' from json_normalize method.
# Path in each object to list of records. If not passed, data will be assumed to 
# be an array of records. If a given field from the JSON stores a nested JSON (or a nested
# dictionary) declare it here to decompose the content of the nested data. e.g. if the field
# 'books' stores a nested JSON, declare, JSON_RECORD_PATH = 'books'

JSON_FIELD_SEPARATOR = "_"
# JSON_FIELD_SEPARATOR = "_" (string). Manipulates the parameter 'sep' from json_normalize method.
# Nested records will generate names separated by sep. 
# e.g., for JSON_FIELD_SEPARATOR = ".", {‘foo’: {‘bar’: 0}} -> foo.bar.
# Then, if a given field 'main_field' stores a nested JSON with fields 'field1', 'field2', ...
# the name of the columns of the dataframe will be formed by concatenating 'main_field', the
# separator, and the names of the nested fields: 'main_field_field1', 'main_field_field2',...

JSON_METADATA_PREFFIX_LIST = None
# JSON_METADATA_PREFFIX_LIST: list of strings (in quotes). Manipulates the parameter 
# 'meta' from json_normalize method. Fields to use as metadata for each record in resulting 
# table. Declare here the non-nested fields, i.e., the fields in the principal JSON. They
# will be repeated in the rows of the dataframe to give the metadata (context) of the rows.

# e.g. Suppose a JSON with the following structure: {'name': 'Mary', 'last': 'Shelley',
# 'books': [{'title': 'Frankestein', 'year': 1818}, {'title': 'Mathilda ', 'year': 1819},{'title': 'The Last Man', 'year': 1826}]},
# Here, there are nested JSONs in the field 'books'. The fields that are not nested
# are 'name' and 'last'.
# Then, JSON_RECORD_PATH = 'books'
# JSON_METADATA_PREFFIX_LIST = ['name', 'last']

#The dataframe will be stored in the object named 'dataset':
# Simply modify this object on the left of equality:
dataset = json_obj_to_dataframe (json_obj_to_convert = JSON_OBJ_TO_CONVERT, json_record_path = JSON_RECORD_PATH, json_field_separator = JSON_FIELD_SEPARATOR, json_metadata_preffix_list = JSON_METADATA_PREFFIX_LIST)

### **Concatenating (SQL UNION) multiple dataframes**

In [ ]:
LIST_OF_DATAFRAMES = [dataset1, dataset2]
# LIST_OF_DATAFRAMES must be a list containing the dataframe objects
# example: list_of_dataframes = [df1, df2, df3, df4]
# Notice that the dataframes are objects, not strings. Therefore, they should not
# be declared inside quotes.
# There is no limit of dataframes. In this example, we will concatenate 4 dataframes.
# If LIST_OF_DATAFRAMES = [df1, df2, df3] we would concatenate 3, and if
# LIST_OF_DATAFRAMES = [df1, df2, df3, df4, df5] we would concatenate 5 dataframes.

WHAT_TO_APPEND = 'rows'
# WHAT_TO_APPEND = 'rows' for appending the rows from one dataframe
# into the other; WHAT_TO_APPEND = 'columns' for appending the columns
# from one dataframe into the other (horizontal or lateral append).

IGNORE_INDEX_ON_UNION = True # Alternatively: True or False

SORT_VALUES_ON_UNION = True # Alternatively: True or False

UNION_JOIN_TYPE = None
# JOIN can be 'inner' to perform an inner join, eliminating the missing values
# The default (None) is 'outer': the dataframes will be stacked on the columns with
# same names but, in case there is no correspondence, the row will present a missing
# value for the columns which are not present in one of the dataframes.
# When using the 'inner' method, only the common columns will remain.
# Alternatively, keep UNION_JOIN_TYPE = None for the standard outer join; or set
# UNION_JOIN_TYPE = "inner" (inside quotes) for using the inner join.
    
#These 3 last parameters are the same from Pandas .concat method:
# IGNORE_INDEX_ON_UNION = ignore_index;
# SORT_VALUES_ON_UNION = sort
# UNION_JOIN_TYPE = join
# Check Datacamp course Joining Data with pandas, Chap.3, 
# Advanced Merging and Concatenating
    

#New dataframe saved as concat_df. Simply modify this object on the left of equality:
concat_df = UNION_DATAFRAMES (list_of_dataframes = LIST_OF_DATAFRAMES, what_to_append = WHAT_TO_APPEND, ignore_index_on_union = IGNORE_INDEX_ON_UNION, sort_values_on_union = SORT_VALUES_ON_UNION, union_join_type = UNION_JOIN_TYPE)

### **Filtering (selecting); or renaming columns of the dataframe**

In [ ]:
DATASET = dataset #Alternatively: object containing the dataset to be analyzed

MODE = 'filter'
# MODE = 'filter' for filtering only the list of columns passed as cols_list;
# MODE = 'rename' for renaming the columns with the names passed as cols_list.

COLS_LIST = ['column1', 'column2', 'column3']
# COLS_LIST = list of strings containing the names (headers) of the columns to select
# (filter); or to be set as the new columns' names, according to the selected mode.
# For instance: COLS_LIST = ['col1', 'col2', 'col3'] will 
# select columns 'col1', 'col2', and 'col3' (or rename the columns with these names). 
# Declare the names inside quotes.
# Simply substitute the list by the list of columns that you want to select; or the
# list of the new names you want to give to the dataset columns.

# New dataframe saved as new_df. Simply modify this object on the left of equality:
new_df = col_filter_rename (df = DATASET, cols_list = COLS_LIST, mode = MODE)

### **Splitting the dataframe into train and test subsets**

In [ ]:
X_df = X
# X_df = subset of predictive variables (dataframe). Alternatively, modify X, not X_df
Y = y
# Y = subset of response variable (series). Alternatively, modify y, not Y

PERCENT_OF_DATA_USED_FOR_MODEL_TRAINING = 75   
# percent_of_data_used_for_model_training: float from 0 to 100,
# representing the percent of data used for training the model

# Subset and series destined to training returned as X_train, y_train;
# Subset and series separated for testing returned as X_test, y_test;
# Simply modify these objects on the left of equality:
X_train, X_test, y_train, y_test = split_data_into_train_and_test (X = X_df, y = Y, percent_of_data_used_for_model_training = PERCENT_OF_DATA_USED_FOR_MODEL_TRAINING)

### **Ordinary Least Squares (OLS) Linear Regression**
- This function runs the 'bar_chart' function. Certify that this function was properly loaded.

In [ ]:
X_TRAIN = X_train
# X_TRAIN = subset of predictive variables (dataframe).
# Alternatively, modify X_train, not X_TRAIN
Y_TRAIN = y_train
# Y_TRAIN = subset of response variable (series).
# Alternatively, modify y_train, not Y_TRAIN

# Model object returned as ols_linear_reg_model;
# Feature ranking dataframe returned as ols_feature_importance_df;
# Simply modify these objects on the left of equality:
ols_linear_reg_model, ols_feature_importance_df = ols_linear_reg (X_train = X_TRAIN, y_train = Y_TRAIN)

### **Ridge Linear Regression**
- This function runs the 'bar_chart' function. Certify that this function was properly loaded.

In [ ]:
X_TRAIN = X_train
# X_TRAIN = subset of predictive variables (dataframe).
# Alternatively, modify X_train, not X_TRAIN
Y_TRAIN = y_train
# Y_TRAIN = subset of response variable (series).
# Alternatively, modify y_train, not Y_TRAIN

ALPHA_HYPERPARAMETER = 1.0
MAXIMUM_OF_ALLOWED_ITERATIONS = 20000
# hyperparameters: alpha = ALPHA_HYPERPARAMETER and MAXIMUM_OF_ALLOWED_ITERATIONS = max_iter

# MAXIMUM_OF_ALLOWED_ITERATIONS = integer representing the maximum number of iterations
# that the optimization algorithm can perform. Depending on data, convergence may not be
# reached within this limit, so you may need to increase this hyperparameter.

# ALPHA_HYPERPARAMETER is the regularization strength and must be a positive float value. 
# Regularization improves the conditioning of the problem and reduces the variance 
# of the estimates. Larger values specify stronger regularization.
# ALPHA_HYPERPARAMETER = 0 is equivalent to an ordinary least square, solved by the 
# LinearRegression object. For numerical reasons, using ALPHA_HYPERPARAMETER = 0 
# is not advised. Given this, you should use the ols_linear_reg function instead.

# Model object returned as ridge_linear_reg_model;
# Feature ranking dataframe returned as ridge_feature_importance_df;
# Simply modify these objects on the left of equality:
ridge_linear_reg_model, ridge_feature_importance_df = ridge_linear_reg (X_train = X_TRAIN, y_train = Y_TRAIN, alpha_hyperparameter = ALPHA_HYPERPARAMETER, maximum_of_allowed_iterations = MAXIMUM_OF_ALLOWED_ITERATIONS)

### **Lasso Linear Regression**
- This function runs the 'bar_chart' function. Certify that this function was properly loaded.

In [ ]:
X_TRAIN = X_train
# X_TRAIN = subset of predictive variables (dataframe).
# Alternatively, modify X_train, not X_TRAIN
Y_TRAIN = y_train
# Y_TRAIN = subset of response variable (series).
# Alternatively, modify y_train, not Y_TRAIN

ALPHA_HYPERPARAMETER = 1.0
MAXIMUM_OF_ALLOWED_ITERATIONS = 20000
# hyperparameters: alpha = ALPHA_HYPERPARAMETER and MAXIMUM_OF_ALLOWED_ITERATIONS = max_iter

# MAXIMUM_OF_ALLOWED_ITERATIONS = integer representing the maximum number of iterations
# that the optimization algorithm can perform. Depending on data, convergence may not be
# reached within this limit, so you may need to increase this hyperparameter.

# ALPHA_HYPERPARAMETER is the regularization strength and must be a positive float value. 
# Regularization improves the conditioning of the problem and reduces the variance 
# of the estimates. Larger values specify stronger regularization.
# ALPHA_HYPERPARAMETER = 0 is equivalent to an ordinary least square, solved by the 
# LinearRegression object. For numerical reasons, using ALPHA_HYPERPARAMETER = 0 
# is not advised. Given this, you should use the ols_linear_reg function instead.

# Model object returned as lasso_linear_reg_model;
# Feature ranking dataframe returned as lasso_feature_importance_df;
# Simply modify these objects on the left of equality:
lasso_linear_reg_model, lasso_feature_importance_df = lasso_linear_reg (X_train = X_TRAIN, y_train = Y_TRAIN, alpha_hyperparameter = ALPHA_HYPERPARAMETER, maximum_of_allowed_iterations = MAXIMUM_OF_ALLOWED_ITERATIONS)

### **Elastic Net Linear Regression**
- This function runs the 'bar_chart' function. Certify that this function was properly loaded.

In [ ]:
X_TRAIN = X_train
# X_TRAIN = subset of predictive variables (dataframe).
# Alternatively, modify X_train, not X_TRAIN
Y_TRAIN = y_train
# Y_TRAIN = subset of response variable (series).
# Alternatively, modify y_train, not Y_TRAIN

ALPHA_HYPERPARAMETER = 1.0
MAXIMUM_OF_ALLOWED_ITERATIONS = 20000
L1_RATIO_HYPERPARAMETER = 0.5
# hyperparameters: alpha = ALPHA_HYPERPARAMETER; MAXIMUM_OF_ALLOWED_ITERATIONS = max_iter
# and L1_RATIO_HYPERPARAMETER = l1_ratio

# MAXIMUM_OF_ALLOWED_ITERATIONS = integer representing the maximum number of iterations
# that the optimization algorithm can perform. Depending on data, convergence may not be
# reached within this limit, so you may need to increase this hyperparameter.

# ALPHA_HYPERPARAMETER is the regularization strength and must be a positive float value. 
# Regularization improves the conditioning of the problem and reduces the variance 
# of the estimates. Larger values specify stronger regularization.

# L1_RATIO_HYPERPARAMETER is The ElasticNet mixing parameter (float), with 0 <= l1_ratio <= 1. 
# For L1_RATIO_HYPERPARAMETER = 0 the penalty is an L2 penalty. For l1_ratio = 1 it is an L1 penalty. 
# For 0 < L1_RATIO_HYPERPARAMETER < 1, the penalty is a combination of L1 and L2.

# ALPHA_HYPERPARAMETER = 0 and L1_RATIO_HYPERPARAMETER = 0 is equivalent to an ordinary 
# least square, solved by the LinearRegression object. For numerical reasons, 
# using ALPHA_HYPERPARAMETER = 0 and L1_RATIO_HYPERPARAMETER = 0 is not advised. 
# Given this, you should use the ols_linear_reg function instead.

# Model object returned as elastic_net_linear_reg_model;
# Feature ranking dataframe returned as elastic_net_feature_importance_df;
# Simply modify these objects on the left of equality:
elastic_net_linear_reg_model, elastic_net_feature_importance_df = elastic_net_linear_reg (X_train = X_TRAIN, y_train = Y_TRAIN, alpha_hyperparameter = ALPHA_HYPERPARAMETER, l1_ratio_hyperparameter = L1_RATIO_HYPERPARAMETER, maximum_of_allowed_iterations = MAXIMUM_OF_ALLOWED_ITERATIONS)

### **Getting a general feature ranking**

In [ ]:
DICTIONARY_OF_FEATURE_RANKINGS_DATAFRAMES = {
    
    'ols_linear_reg': ols_feature_importance_df,
    'ridge_linear_reg': ridge_feature_importance_df,
    'lasso_linear_reg': lasso_feature_importance_df,
    'elastic_net_linear_reg': elastic_net_feature_importance_df
    
}
# DICTIONARY_OF_FEATURE_RANKINGS_DATAFRAMES
# The key of this dictionary must be the model name or the name of the ranking.
# This key will be used to identify the column on the new dataframe (it will be
# used as suffix). The correspondent value must be the feature importance ranking
# dataframe, with configuration similar to reg_dict: it must have a column
# named, 'predictive_features', which will be used as key for merging.
# For instance, for a dictionary = {'ols_linear_regression': ols_feature_df,
# 'ridge_linear_regression': ridge_feature_df}, the columns 'regression_coefficients'
# will be identified as: 'regression_coefficients_ols_linear_regression' and
# 'regression_coefficients_ridge_linear_regression'. Notice that the underscore ("_")
# is used as suffix separator.

ELIMINATE_NON_CORRESPONDENCE = False
# ELIMINATE_NON_CORRESPONDENCE = False. Since the dataframes will be merged using an
# "outer" join, all entries from all dataframes will be added, possibly resulting in
# missing values (pandas "outer" is a full outer join). 
# Then, set ELIMINATE_NON_CORRESPONDENCE = True to eliminate all missing values
# (rows with entries without correspondence).
LIMIT_OF_RANKED_FEATURES = None
# LIMIT_OF_RANKED_FEATURES = None. Alternatively, set as an integer to limit the number
# of ranked features. e.g. LIMIT_OF_RANKED_FEATURES = 20 will return a dataset with only
# 20 features. Notice that the features are sorted in accordance to their order in the
# input dictionary. Then, the most important ranking will be the one from the first dataframe.

# General feature ranking dataframe returned as general_feature_ranking_df;
# Simply modify this object on the left of equality:
general_feature_ranking_df = (dictionary_of_feature_rankings_dataframes = DICTIONARY_OF_FEATURE_RANKINGS_DATAFRAMES, eliminate_non_correspondence = ELIMINATE_NON_CORRESPONDENCE, limit_of_ranked_features = LIMIT_OF_RANKED_FEATURES)

### **Calculating metrics for regression models**

In [ ]:
MODEL_OBJECT = ols_linear_reg_model # Alternatively: object storing another model
# MODEL_OBJECT: object containing the model that will be analyzed. e.g.
# MODEL_OBJECT = elastic_net_linear_reg_model

X_TRAIN = X_train
# X_TRAIN = subset of predictive variables (dataframe).
# Alternatively, modify X_train, not X_TRAIN
Y_TRAIN = y_train
# Y_TRAIN = subset of response variable (series).
# Alternatively, modify y_train, not Y_TRAIN

X_TEST = None
# X_TEST = subset of predictive variables (test dataframe, in case the original one was 
# split into train and test). Alternatively, modify X_test (or None), not X_TEST
# e.g. X_TEST = X_test
Y_TEST = None
# Y_TEST = subset of response variable (test series, in case the original one was 
# split into train and test). Alternatively, modify y_test (or None), not Y_TEST
# e.g. Y_TEST = y_test

# Dictionary containing calculated metrics returned as metrics_dict;
# Simply modify this object on the left of equality:
metrics_dict = regression_models_metrics (model_object = MODEL_OBJECT, X_train = X_TRAIN, y_train = Y_TRAIN, X_test = X_TEST, y_test = Y_TEST)

### **Making predictions with the models**

In [ ]:
MODEL_OBJECT = ols_linear_reg_model # Alternatively: object storing another model
# MODEL_OBJECT: object containing the model that will be analyzed. e.g.
# MODEL_OBJECT = elastic_net_linear_reg_model

PREDICT_FOR = 'subset'
# Alternatively: PREDICT_FOR = 'subset' or PREDICT_FOR = 'single_entry'

X_df = X
# X_df = subset of predictive variables (dataframe). Alternatively, modify X, not X_df
# If PREDICT_FOR = 'single_entry', X should be a list of parameters values.
# e.g. X = [1.2, 3, 4] (dot is the decimal case separator, comma separate values). 
# Notice that the list should contain only the numeric values, in the same order of the
# correspondent columns.
# If PREDICT_FOR = 'subset' (prediction for multiple entries), X should be a dataframe 
# (subset) of the parameters values, as usual.

CONCATENATE_PREDICTIONS_WITH_X = True
# CONCATENATE_PREDICTIONS_WITH_X = True to concatenate the predicted values with X. 
# For a single entry, it will be appended as an element from the list. For multiple entries, 
# it will be added as a column.
# If CONCATENATE_PREDICTIONS_WITH_X = False, the prediction will be returned as a single variable value
# (single entry), or as a series (prediction for a subset).

COLUMN_WITH_PREDICTIONS_SUFFIX = None
# COLUMN_WITH_PREDICTIONS_SUFFIX = None. If you want to add a new column, you can declare this
# parameter as string with a suffix for identifying the model. If no suffix is added, the new
# column will be named 'y_pred'.
# e.g. COLUMN_WITH_PREDICTIONS_SUFFIX = '_keras' will create a column named "y_pred_keras". This
# parameter is useful when working with multiple models. Always start the suffix with underscore
# "_" so that no blank spaces are added; the suffix will not be merged to the column; and there
# will be no confusion with the dot (.) notation for methods, JSON attributes, etc.

# Prediction (single float value or series of values if CONCATENATE_PREDICTIONS_WITH_X = False);
# (list appended with the prediction or subset with a new column with predictions if 
# CONCATENATE_PREDICTIONS_WITH_X = True) returned as prediction_output
# Simply modify this object (or variable) on the left of equality:
prediction_output = make_model_predictions (model_object = MODEL_OBJECT, X = X_df, predict_for = PREDICT_FOR, concatenate_predictions_with_X = CONCATENATE_PREDICTIONS_WITH_X, col_with_predictions_suffix = COLUMN_WITH_PREDICTIONS_SUFFIX)

### **Importing or exporting models and dictionaries**

#### Case 1: import only a model

In [ ]:
ACTION = 'import'
# ACTION = 'import' for importing a model and/or a dictionary;
# ACTION = 'export' for exporting a model and/or a dictionary.

OBJECTS_MANIPULATED = 'model_only'
# OBJECTS_MANIPULATED = 'model_only' if only a model will be manipulated.
# OBJECTS_MANIPULATED = 'dict_only' if only a dictionary will be manipulated.
# OBJECTS_MANIPULATED = 'model_and_dict' if both a model and a dictionary will 
#  be manipulated.

MODEL_FILE_NAME = None
# MODEL_FILE_NAME: string with the name of the file containing the model (for 'import');
# or of the name that the exported file will have (for 'export')
# e.g. MODEL_FILE_NAME = 'model'
# WARNING: Do not add the file extension.
# Keep it in quotes. Keep MODEL_FILE_NAME = None if no model will be manipulated.

DICTIONARY_FILE_NAME = None
# DICTIONARY_FILE_NAME: string with the name of the file containing the dictionary 
# (for 'import');
# or of the name that the exported file will have (for 'export')
# e.g. DICTIONARY_FILE_NAME = 'history_dict'
# WARNING: Do not add the file extension.
# Keep it in quotes. Keep DICTIONARY_FILE_NAME = None if no dictionary will be manipulated.

DIRECTORY_PATH = '/'
# DIRECTORY_PATH: path of the directory where the model will be saved,
# or from which the model will be retrieved. If no value is provided,
# the DIRECTORY_PATH will be the root: "/"
# Notice that the model and the dictionary must be stored in the same path.
# If a model and a dictionary will be exported, they will be stored in the same
# DIRECTORY_PATH.
    
MODEL_TYPE = 'sklearn'
# This parameter has effect only when a model will be manipulated.
# MODEL_TYPE: 'keras' for deep learning keras/ tensorflow models with extension .h5
# MODEL_TYPE = 'sklearn' for models from scikit-learn (non-deep learning)
# MODEL_TYPE = 'xgb' for XGBoost models (non-deep learning)
# MODEL_TYPE = 'arima' for ARIMA model (Statsmodels)

DICT_TO_EXPORT = None
MODEL_TO_EXPORT = None 
# These two parameters have effect only when ACTION == 'export'. In this case, they
# must be declared. If ACTION == 'export', keep:
# DICT_TO_EXPORT = None, 
# MODEL_TO_EXPORT = None
# If one of these objects will be exported, substitute None by the name of the object
# e.g. if your model is stored in the global memory as 'keras_model' declare:
# MODEL_TO_EXPORT = keras_model. Notice that it must be declared without quotes, since
# it is not a string, but an object.
# For exporting a dictionary named as 'dict':
# DICT_TO_EXPORT = dict

USE_COLAB_MEMORY = False
# USE_COLAB_MEMORY: this parameter has only effect when using Google Colab (or it will
# raise an error). Set as USE_COLAB_MEMORY = True if you want to use the instant memory
# from Google Colaboratory: you will update or download the file and it will be available
# only during the time when the kernel is running. It will be excluded when the kernel
# dies, for instance, when you close the notebook.
    
# If ACTION == 'export' and USE_COLAB_MEMORY == True, then the file will be downloaded
# to your computer (running the cell will start the download).

#Model object saved as model.
# Simply modify this object on the left of equality:
model = import_export_model_or_dict (action = ACTION, objects_manipulated = OBJECTS_MANIPULATED, model_file_name = MODEL_FILE_NAME, dictionary_file_name = DICTIONARY_FILE_NAME, directory_path = DIRECTORY_PATH, model_type = MODEL_TYPE, dict_to_export = DICT_TO_EXPORT, model_to_export = MODEL_TO_EXPORT, use_colab_memory = USE_COLAB_MEMORY)    

#### Case 2: import only a dictionary

In [ ]:
ACTION = 'import'
# ACTION = 'import' for importing a model and/or a dictionary;
# ACTION = 'export' for exporting a model and/or a dictionary.

OBJECTS_MANIPULATED = 'dict_only'
# OBJECTS_MANIPULATED = 'model_only' if only a model will be manipulated.
# OBJECTS_MANIPULATED = 'dict_only' if only a dictionary will be manipulated.
# OBJECTS_MANIPULATED = 'model_and_dict' if both a model and a dictionary will 
#  be manipulated.

MODEL_FILE_NAME = None
# MODEL_FILE_NAME: string with the name of the file containing the model (for 'import');
# or of the name that the exported file will have (for 'export')
# e.g. MODEL_FILE_NAME = 'model'
# WARNING: Do not add the file extension.
# Keep it in quotes. Keep MODEL_FILE_NAME = None if no model will be manipulated.

DICTIONARY_FILE_NAME = None
# DICTIONARY_FILE_NAME: string with the name of the file containing the dictionary 
# (for 'import');
# or of the name that the exported file will have (for 'export')
# e.g. DICTIONARY_FILE_NAME = 'history_dict'
# WARNING: Do not add the file extension.
# Keep it in quotes. Keep DICTIONARY_FILE_NAME = None if no dictionary will be manipulated.

DIRECTORY_PATH = '/'
# DIRECTORY_PATH: path of the directory where the model will be saved,
# or from which the model will be retrieved. If no value is provided,
# the DIRECTORY_PATH will be the root: "/"
# Notice that the model and the dictionary must be stored in the same path.
# If a model and a dictionary will be exported, they will be stored in the same
# DIRECTORY_PATH.
    
MODEL_TYPE = 'sklearn'
# This parameter has effect only when a model will be manipulated.
# MODEL_TYPE: 'keras' for deep learning keras/ tensorflow models with extension .h5
# MODEL_TYPE = 'sklearn' for models from scikit-learn (non-deep learning)
# MODEL_TYPE = 'xgb' for XGBoost models (non-deep learning)
# MODEL_TYPE = 'arima' for ARIMA model (Statsmodels)

DICT_TO_EXPORT = None
MODEL_TO_EXPORT = None 
# These two parameters have effect only when ACTION == 'export'. In this case, they
# must be declared. If ACTION == 'export', keep:
# DICT_TO_EXPORT = None, 
# MODEL_TO_EXPORT = None
# If one of these objects will be exported, substitute None by the name of the object
# e.g. if your model is stored in the global memory as 'keras_model' declare:
# MODEL_TO_EXPORT = keras_model. Notice that it must be declared without quotes, since
# it is not a string, but an object.
# For exporting a dictionary named as 'dict':
# DICT_TO_EXPORT = dict

USE_COLAB_MEMORY = False
# USE_COLAB_MEMORY: this parameter has only effect when using Google Colab (or it will
# raise an error). Set as USE_COLAB_MEMORY = True if you want to use the instant memory
# from Google Colaboratory: you will update or download the file and it will be available
# only during the time when the kernel is running. It will be excluded when the kernel
# dies, for instance, when you close the notebook.
    
# If ACTION == 'export' and USE_COLAB_MEMORY == True, then the file will be downloaded
# to your computer (running the cell will start the download).

# Dictionary saved as imported_dict.
# Simply modify this object on the left of equality:
imported_dict = import_export_model_or_dict (action = ACTION, objects_manipulated = OBJECTS_MANIPULATED, model_file_name = MODEL_FILE_NAME, dictionary_file_name = DICTIONARY_FILE_NAME, directory_path = DIRECTORY_PATH, model_type = MODEL_TYPE, dict_to_export = DICT_TO_EXPORT, model_to_export = MODEL_TO_EXPORT, use_colab_memory = USE_COLAB_MEMORY)    

#### Case 3: import a model and a dictionary

In [ ]:
ACTION = 'import'
# ACTION = 'import' for importing a model and/or a dictionary;
# ACTION = 'export' for exporting a model and/or a dictionary.

OBJECTS_MANIPULATED = 'model_and_dict'
# OBJECTS_MANIPULATED = 'model_only' if only a model will be manipulated.
# OBJECTS_MANIPULATED = 'dict_only' if only a dictionary will be manipulated.
# OBJECTS_MANIPULATED = 'model_and_dict' if both a model and a dictionary will 
#  be manipulated.

MODEL_FILE_NAME = None
# MODEL_FILE_NAME: string with the name of the file containing the model (for 'import');
# or of the name that the exported file will have (for 'export')
# e.g. MODEL_FILE_NAME = 'model'
# WARNING: Do not add the file extension.
# Keep it in quotes. Keep MODEL_FILE_NAME = None if no model will be manipulated.

DICTIONARY_FILE_NAME = None
# DICTIONARY_FILE_NAME: string with the name of the file containing the dictionary 
# (for 'import');
# or of the name that the exported file will have (for 'export')
# e.g. DICTIONARY_FILE_NAME = 'history_dict'
# WARNING: Do not add the file extension.
# Keep it in quotes. Keep DICTIONARY_FILE_NAME = None if no dictionary will be manipulated.

DIRECTORY_PATH = '/'
# DIRECTORY_PATH: path of the directory where the model will be saved,
# or from which the model will be retrieved. If no value is provided,
# the DIRECTORY_PATH will be the root: "/"
# Notice that the model and the dictionary must be stored in the same path.
# If a model and a dictionary will be exported, they will be stored in the same
# DIRECTORY_PATH.
    
MODEL_TYPE = 'sklearn'
# This parameter has effect only when a model will be manipulated.
# MODEL_TYPE: 'keras' for deep learning keras/ tensorflow models with extension .h5
# MODEL_TYPE = 'sklearn' for models from scikit-learn (non-deep learning)
# MODEL_TYPE = 'xgb' for XGBoost models (non-deep learning)
# MODEL_TYPE = 'arima' for ARIMA model (Statsmodels)

DICT_TO_EXPORT = None
MODEL_TO_EXPORT = None 
# These two parameters have effect only when ACTION == 'export'. In this case, they
# must be declared. If ACTION == 'export', keep:
# DICT_TO_EXPORT = None, 
# MODEL_TO_EXPORT = None
# If one of these objects will be exported, substitute None by the name of the object
# e.g. if your model is stored in the global memory as 'keras_model' declare:
# MODEL_TO_EXPORT = keras_model. Notice that it must be declared without quotes, since
# it is not a string, but an object.
# For exporting a dictionary named as 'dict':
# DICT_TO_EXPORT = dict

USE_COLAB_MEMORY = False
# USE_COLAB_MEMORY: this parameter has only effect when using Google Colab (or it will
# raise an error). Set as USE_COLAB_MEMORY = True if you want to use the instant memory
# from Google Colaboratory: you will update or download the file and it will be available
# only during the time when the kernel is running. It will be excluded when the kernel
# dies, for instance, when you close the notebook.
    
# If ACTION == 'export' and USE_COLAB_MEMORY == True, then the file will be downloaded
# to your computer (running the cell will start the download).

# Model object saved as model. Dictionary saved as imported_dict.
# Simply modify these objects on the left of equality:
model, imported_dict = import_export_model_or_dict (action = ACTION, objects_manipulated = OBJECTS_MANIPULATED, model_file_name = MODEL_FILE_NAME, dictionary_file_name = DICTIONARY_FILE_NAME, directory_path = DIRECTORY_PATH, model_type = MODEL_TYPE, dict_to_export = DICT_TO_EXPORT, model_to_export = MODEL_TO_EXPORT, use_colab_memory = USE_COLAB_MEMORY)    

#### Case 4: export a model and/or a dictionary

In [ ]:
ACTION = 'export'
# ACTION = 'import' for importing a model and/or a dictionary;
# ACTION = 'export' for exporting a model and/or a dictionary.

OBJECTS_MANIPULATED = 'model_only'
# OBJECTS_MANIPULATED = 'model_only' if only a model will be manipulated.
# OBJECTS_MANIPULATED = 'dict_only' if only a dictionary will be manipulated.
# OBJECTS_MANIPULATED = 'model_and_dict' if both a model and a dictionary will 
#  be manipulated.

MODEL_FILE_NAME = None
# MODEL_FILE_NAME: string with the name of the file containing the model (for 'import');
# or of the name that the exported file will have (for 'export')
# e.g. MODEL_FILE_NAME = 'model'
# WARNING: Do not add the file extension.
# Keep it in quotes. Keep MODEL_FILE_NAME = None if no model will be manipulated.

DICTIONARY_FILE_NAME = None
# DICTIONARY_FILE_NAME: string with the name of the file containing the dictionary 
# (for 'import');
# or of the name that the exported file will have (for 'export')
# e.g. DICTIONARY_FILE_NAME = 'history_dict'
# WARNING: Do not add the file extension.
# Keep it in quotes. Keep DICTIONARY_FILE_NAME = None if no dictionary will be manipulated.

DIRECTORY_PATH = '/'
# DIRECTORY_PATH: path of the directory where the model will be saved,
# or from which the model will be retrieved. If no value is provided,
# the DIRECTORY_PATH will be the root: "/"
# Notice that the model and the dictionary must be stored in the same path.
# If a model and a dictionary will be exported, they will be stored in the same
# DIRECTORY_PATH.
    
MODEL_TYPE = 'sklearn'
# This parameter has effect only when a model will be manipulated.
# MODEL_TYPE: 'keras' for deep learning keras/ tensorflow models with extension .h5
# MODEL_TYPE = 'sklearn' for models from scikit-learn (non-deep learning)
# MODEL_TYPE = 'xgb' for XGBoost models (non-deep learning)
# MODEL_TYPE = 'arima' for ARIMA model (Statsmodels)

DICT_TO_EXPORT = None
MODEL_TO_EXPORT = None 
# These two parameters have effect only when ACTION == 'export'. In this case, they
# must be declared. If ACTION == 'export', keep:
# DICT_TO_EXPORT = None, 
# MODEL_TO_EXPORT = None
# If one of these objects will be exported, substitute None by the name of the object
# e.g. if your model is stored in the global memory as 'keras_model' declare:
# MODEL_TO_EXPORT = keras_model. Notice that it must be declared without quotes, since
# it is not a string, but an object.
# For exporting a dictionary named as 'dict':
# DICT_TO_EXPORT = dict

USE_COLAB_MEMORY = False
# USE_COLAB_MEMORY: this parameter has only effect when using Google Colab (or it will
# raise an error). Set as USE_COLAB_MEMORY = True if you want to use the instant memory
# from Google Colaboratory: you will update or download the file and it will be available
# only during the time when the kernel is running. It will be excluded when the kernel
# dies, for instance, when you close the notebook.
    
# If ACTION == 'export' and USE_COLAB_MEMORY == True, then the file will be downloaded
# to your computer (running the cell will start the download).

import_export_model_or_dict (action = ACTION, objects_manipulated = OBJECTS_MANIPULATED, model_file_name = MODEL_FILE_NAME, dictionary_file_name = DICTIONARY_FILE_NAME, directory_path = DIRECTORY_PATH, model_type = MODEL_TYPE, dict_to_export = DICT_TO_EXPORT, model_to_export = MODEL_TO_EXPORT, use_colab_memory = USE_COLAB_MEMORY)    

## **Exporting the dataframe as CSV file**

In [ ]:
## WARNING: all file extensions should be .csv for this function

DATAFRAME_TO_BE_EXPORTED = dataset
#Alternatively: object containing the dataset to be exported.

FILE_DIRECTORY_PATH = "/"
# FILE_DIRECTORY_PATH - (string, in quotes): input the path of the directory 
# (e.g. folder path) where the file is stored. e.g. FILE_DIRECTORY_PATH = "/" 
# or FILE_DIRECTORY_PATH = "/folder"
# If you want to export the file to AWS S3, this parameter will have no effect.
# In this case, you can set FILE_DIRECTORY_PATH = None

NEW_FILE_NAME_WITH_CSV_EXTENSION = "dataset.csv"
# NEW_FILE_NAME_WITH_CSV_EXTENSION - (string, in quotes): input the name of the file with the 
# extension. e.g. FILE_NAME_WITH_CSV_EXTENSION = "file.csv"

EXPORT_TO_S3_BUCKET = False
# export_to_s3_bucket = False. Alternatively, set as True to export the file to an
# AWS S3 Bucket.
    
## The following parameters have effect only when export_to_s3_bucket == True:

S3_BUCKET_NAME = None    
## This parameter is obbligatory to access an AWS S3 bucket. Substitute it for a string
# with the bucket's name. e.g. S3_BUCKET_NAME = "aws-bucket-1" access a bucket named as
# "aws-bucket-1"
DESIRED_S3_FILE_NAME_WITH_CSV_EXTENSION = None
# The name desired for the object stored in S3 (string, in quotes). 
# Keep it None to set it equals to NEW_FILE_NAME_WITH_CSV_EXTENSION. 
# Alternatively, set it as a string analogous to NEW_FILE_NAME_WITH_CSV_EXTENSION.
# e.g. DESIRED_S3_FILE_NAME_WITH_CSV_EXTENSION = "S3_file.csv"

export_dataframe(dataframe_to_be_exported = DATAFRAME_TO_BE_EXPORTED, new_file_name_with_csv_extension = NEW_FILE_NAME_WITH_CSV_EXTENSION, file_directory_path = FILE_DIRECTORY_PATH, export_to_s3_bucket = EXPORT_TO_S3_BUCKET, s3_bucket_name = S3_BUCKET_NAME, desired_s3_file_name_with_csv_extension = DESIRED_S3_FILE_NAME_WITH_CSV_EXTENSION)

## **Downloading a file from Google Colab or AWS S3 to the local machine or uploading a file from the machine to S3 or to Colab's instant memory**

In [ ]:
SOURCE = 'aws'
# SOURCE = 'google' for downloading from (or uploading to) Google Colab's instant memory;
# SOURCE = 'aws' for downloading from (or uploading to) an AWS S3 bucket.

ACTION = 'download'
# ACTION = 'download' to download the file to the local machine
# ACTION = 'upload' to upload a file from local machine to AWS S3 or to Google Colab's 
# instant memory

OBJECT_TO_DOWNLOAD_FROM_COLAB = None
# OBJECT_TO_DOWNLOAD_FROM_COLAB = None. This option has effect only when
# SOURCE == 'google'. In this case, this parameter is obbligatory. 
# Declare as OBJECT_TO_DOWNLOAD_FROM_COLAB the object that you want to download.
# Since it is an object and not a string, it should not be declared in quotes.
# e.g. to download a dictionary named dict, OBJECT_TO_DOWNLOAD_FROM_COLAB = dict.
# To download a dataframe named df, declare OBJECT_TO_DOWNLOAD_FROM_COLAB = df.
# To export a model named keras_model, declare OBJECT_TO_DOWNLOAD_FROM_COLAB = keras_model
    
## THE FOLLOWING PARAMETERS HAVE EFFECT ONLY WHEN source == 'aws'

S3_BUCKET_NAME = None
## This parameter is obbligatory to access an AWS S3 bucket. Substitute it for a string
# with the bucket's name. e.g. S3_BUCKET_NAME = "aws-bucket-1" access a bucket named as
# "aws-bucket-1"

LOCAL_PATH_OF_STORAGE = '/'
# LOCAL_PATH_OF_STORAGE: path of the local computer environment 
# to which the S3 bucket contents will be downloaded (ACTION == 'download'); or
# path of the folder containing the file that will be uploaded in S3 (ACTION = 'upload'). 
# If it is None, or if LOCAL_PATH_OF_STORAGE = '/', files 
# will be imported to the root path. Alternatively, input the path as a string (in quotes). 
# Examples: LOCAL_PATH_OF_STORAGE = '/copied_s3_bucket'; 
# LOCAL_PATH_OF_STORAGE = "/My_folder"; LOCAL_PATH_OF_STORAGE = "/Users/Me/Documents/"
# Notice that only the directories should be declared: do not include the file name and
# its extension.

FILE_NAME_WITH_EXTENSION = None
# FILE_NAME_WITH_EXTENSION: string, in quotes, containing the file name which will be
# downloaded from S3; or uploaded from S3, followed by its extension. 
## This parameter is obbligatory when SOURCE == 'aws'
# Examples:
# FILE_NAME_WITH_EXTENSION = 'Screen_Shot.png'; FILE_NAME_WITH_EXTENSION = 'dataset.csv',
# FILE_NAME_WITH_EXTENSION = "dictionary.pkl", FILE_NAME_WITH_EXTENSION = "model.h5",
# FILE_NAME_WITH_EXTENSION = 'doc.pdf', FILE_NAME_WITH_EXTENSION = 'model.dill'

download_or_upload_file (source = SOURCE, action = ACTION, object_to_download_from_colab = OBJECT_TO_DOWNLOAD_FROM_COLAB, s3_bucket_name = S3_BUCKET_NAME, local_path_of_storage = LOCAL_PATH_OF_STORAGE, file_name_with_extension = FILE_NAME_WITH_EXTENSION)

### **Plotting a bar chart**
- Bars may be vertically or horizontally oriented.
- Bar charts are plotted after selecting an aggregation function, and the cumulative percent curve may be obtained and plotted with the bars (in secondary axis).
- To obtain a Pareto chart, keep aggregate_function = 'sum', plot_cumulative_percent = True, and orientation = 'vertical'.

In [ ]:
DATASET = dataset #Alternatively: object containing the dataset to be analyzed

CATEGORICAL_VAR_NAME = 'categorical_column_name'
# CATEGORICAL_VAR_NAME: string (inside quotes) containing the name 
# of the column to be analyzed. e.g. 
# CATEGORICAL_VAR_NAME = "column1"

RESPONSE_VAR_NAME = "response_column_name"
# RESPONSE_VAR_NAME: string (inside quotes) containing the name 
# of the column that stores the response correspondent to the
# categories. e.g. RESPONSE_VAR_NAME = "response_feature"

AGGREGATE_FUNCTION = 'sum'
# AGGREGATE_FUNCTION = 'sum': String defining the aggregation 
# method that will be applied. Possible values:
# 'median', 'mean', 'mode', 'sum', 'min', 'max', 'variance',
# 'standard_deviation','10_percent_quantile', '20_percent_quantile',
# '25_percent_quantile', '30_percent_quantile', '40_percent_quantile',
# '50_percent_quantile', '60_percent_quantile', '70_percent_quantile',
# '75_percent_quantile', '80_percent_quantile', '90_percent_quantile',
# and '95_percent_quantile'.
# To use another aggregate function, the method must be added to the
# dictionary of methods agg_methods_dict, defined in the function.
# If None or an invalid function is input, 'sum' will be used.

ADD_SUFFIX_TO_AGGREGATED_COL = True
# ADD_SUFFIX_TO_AGGREGATED_COL = True will add a suffix to the
# aggregated column. e.g. 'responseVar_mean'. If ADD_SUFFIX_TO_AGGREGATED_COL
# = False, the aggregated column will have the original column name.
SUFFIX = None
# suffix = None. Keep it None if no suffix should be added, or if
# the name of the aggregate function should be used as suffix, after
# "_". Alternatively, set it as a string. As recommendation, put the
# "_" sign in the beginning of this string to separate the suffix from
# the original column name. e.g. if the response variable is 'Y' and
# suffix = '_agg', the new aggregated column will be named as 'Y_agg'
CALCULATE_AND_PLOT_CUMULATIVE_PERCENT = True
# CALCULATE_AND_PLOT_CUMULATIVE_PERCENT = True to calculate and plot
# the line of cumulative percent, or 
# CALCULATE_AND_PLOT_CUMULATIVE_PERCENT = False to omit it.
# This feature is only shown when AGGREGATE_FUNCTION = 'sum', 'median',
# 'mean', or 'mode'. So, it will be automatically set as False if 
# another aggregate is selected.
ORIENTATION = 'vertical'
# ORIENTATION = 'vertical' is the standard, and plots vertical bars
# (perpendicular to the X axis). In this case, the categories are shown
# in the X axis, and the correspondent responses are in Y axis.
# Alternatively, ORIENTATION = 'horizontal' results in horizontal bars.
# In this case, categories are in Y axis, and responses in X axis.
# If None or invalid values are provided, orientation is set as 'vertical'.
LIMIT_OF_PLOTTED_CATEGORIES = None
# LIMIT_OF_PLOTTED_CATEGORIES: integer value that represents
# the maximum of categories that will be plot. Keep it None to plot
# all categories. Alternatively, set an integer value. e.g.: if
# LIMIT_OF_PLOTTED_CATEGORIES = 4, but there are more categories,
# the dataset will be sorted in descending order and: 1) The remaining
# categories will be sum in a new category named 'others' if the
# aggregate function is 'sum'; 2) Or the other categories will be simply
# omitted from the plot, for other aggregate functions. Notice that
# it limits only the variables in the plot: all of them will be
# returned in the dataframe.
# Use this parameter to obtain a cleaner plot. Notice that the remaining
# columns will be aggregated as 'others' even if there is a single column
# beyond the limit.

X_AXIS_ROTATION = 70
#Rotation of X axis labels. Alternatively, insert any numeric value from 0 to 90 (degrees).
Y_AXIS_ROTATION = 0
#Rotation of Y axis labels. Alternatively, insert any numeric value from 0 to 90 (degrees).

GRID = True #Alternatively: True or False
# If GRID = False, no grid lines are shown in the graphic.

HORIZONTAL_AXIS_TITLE = None #Alternatively: string inside quotes for horizontal title
VERTICAL_AXIS_TITLE = None #Alternatively: string inside quotes for vertical title
PLOT_TITLE = None #Alternatively: string inside quotes for graphic title
# e.g. HORIZONTAL_AXIS_TITLE = "X", VERTICAL_AXIS_TITLE = "Y", PLOT_TITLE = "YxX"

EXPORT_PNG = False
# Set EXPORT_PNG = False if you do not want to export the obtained image;
# Set EXPORT_PNG = True to export the obtained image.

DIRECTORY_TO_SAVE = None
# This parameter has effect only if EXPORT_PNG = True.
# (string, in quotes): input the path of the directory 
# (e.g. folder path) where the file will be stored. e.g. DIRECTORY_TO_SAVE = "/" 
# or DIRECTORY_TO_SAVE = "/folder"
# If EXPORT_PNG = True and DIRECTORY_TO_SAVE = None, the file will be saved in the root
# path, DIRECTORY_TO_SAVE = "/"

FILE_NAME = None
# This parameter has effect only if EXPORT_PNG = True.
# (string, in quotes): input the name you want for the file without the 
# extension, which will be 'png'. e.g. FILE_NAME = "my_image" will save a file 'my_image.png' 
# If EXPORT_PNG = True and FILE_NAME = None, the file will be saved as:
# 'scatter_plot_lin_reg.png'
# WARNING: if there is already a file in the path DIRECTORY_TO_SAVE saved as FILE_NAME,
# the file will be overwritten.

PNG_RESOLUTION_DPI = 110
# This parameter has effect only if EXPORT_PNG = True.
# Alternatively, input an integer that will correspond to the resolution of the exported
# image in dpi. If PNG_RESOLUTION_DPI = None, it will be set as 110.

# New dataframe saved as aggregated_sorted_df. 
# Simply modify this object on the left of equality:
aggregated_sorted_df = bar_chart (df = DATASET, categorical_var_name = CATEGORICAL_VAR_NAME, response_var_name = RESPONSE_VAR_NAME, aggregate_function = AGGREGATE_FUNCTION, add_suffix_to_aggregated_col = ADD_SUFFIX_TO_AGGREGATED_COL, suffix = SUFFIX, calculate_and_plot_cumulative_percent = CALCULATE_AND_PLOT_CUMULATIVE_PERCENT, orientation = ORIENTATION, limit_of_plotted_categories = LIMIT_OF_PLOTTED_CATEGORIES, horizontal_axis_title = HORIZONTAL_AXIS_TITLE, vertical_axis_title = VERTICAL_AXIS_TITLE, plot_title = PLOT_TITLE, x_axis_rotation = X_AXIS_ROTATION, y_axis_rotation = Y_AXIS_ROTATION, grid = GRID, export_png = EXPORT_PNG, directory_to_save = DIRECTORY_TO_SAVE, file_name = FILE_NAME, png_resolution_dpi = PNG_RESOLUTION_DPI)

****